# Cumulative Modified HRUs between Pathways

Two pathways are used within the framework.

1. H0: null hypothesis of historical weather statistics
2. H1: alternative hypothesis of weather statistics extracted from down-scaled, global climate model results.

There are four analysis periods

1. Data Period: 1981-2010
2. Projection Period 1: 2011-2040
3. Projection Period 2: 2041-2070
4. Projection Period 3: 2071-2100

## Imports and Parameters

In [1]:
%matplotlib inline

In [2]:
import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
from math import floor, ceil
from copy import deepcopy

In [3]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [4]:
# input directories and file roots
IN_DIR = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Recharge\NumModel\HSPF\Basin_10P_10K'
OUT_DIR = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Recharge\NumModel\HSPF\Basin_10P_10K\Processed'
OUT_ROOT = "DCmHBas_HRUs_"
NUMREAL = 10000
IN_RAET0 = "R%d_H0_AET_DF.pickle"
IN_RAET1 = "R%d_H1_AET_DF.pickle"
IN_RRE0 = "R%d_H0_Re_DF.pickle"
IN_RRE1 = "R%d_H1_Re_DF.pickle"
IN_RRO0 = "R%d_H0_RO_DF.pickle"
IN_RRO1 = "R%d_H1_RO_DF.pickle"

In [5]:
# time periods
TP_DICT = { 1 : [ dt.datetime(1981, 1, 1), dt.datetime(2010, 12, 31)],
            2 : [ dt.datetime(2011, 1, 1), dt.datetime(2040, 12, 31)],
            3 : [ dt.datetime(2041, 1, 1), dt.datetime(2070, 12, 31)],
            4 : [ dt.datetime(2071, 1, 1), dt.datetime(2100, 12, 31)],}

In [6]:
OurQs = np.array( [ round( 0.01 * x, 2 ) for x in range(101) ], dtype=np.float32 )
NumQs = len( OurQs )
NumQs

101

In [7]:
StatsHdsList = list()
for iI in range(NumQs):
    cQ = OurQs[iI]
    Hdr = "%d_ptile" % round(cQ * 100.0)
    StatsHdsList.append( Hdr )
# end of for
StatsHdsList.append( "Average" )
StatsHdsList.append( "Variance" )
StatsHdsList.append( "Skew" )
StatsHdsList.append( "Kurt" )

In [8]:
NumHdrs = len( StatsHdsList )
NumHdrs

105

In [9]:
HRU_Labels = [ "HRU_1", "HRU_2" ]
RR_Labels = [ "R001", "R002", "R003", "R004", "R005" ]

In [10]:
AET_In_Hdrs = ["HRU_1", 
               "HRU_2", 
               "R001", 
               "R002", 
               "R003", 
               "R004", 
               "R005" ]
RE_In_Hdrs = ["HRU_1", 
               "HRU_2", 
               "R001", 
               "R002", 
               "R003", 
               "R004" ]
RO_In_Hdrs = ["HRU_1", 
               "HRU_2", 
               "R001", 
               "R002", 
               "R003", 
               "R004", 
               "R005" ]

## Collate DataFrames

### H0

Go through all realizations and create a DataFrame for each constituent and each period

In [11]:
for rR in range( 1, NUMREAL + 1, 1 ):
    if ((rR % 200) == 0):
        print("Working on realization %d" % rR)
    # end if
    # AET
    InFile = IN_RAET0 % rR
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    InDF = InDF[AET_In_Hdrs].copy()
    H0_AET = InDF.resample( 'MS', closed='left', label='left' ).sum()
    # RO
    InFile = IN_RRO0 % rR
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    InDF = InDF[RO_In_Hdrs].copy()
    H0_RO = InDF.resample( 'MS', closed='left', label='left' ).sum()
    # Re
    InFile = IN_RRE0 % rR
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    InDF = InDF[RE_In_Hdrs].copy()
    H0_RE = InDF.resample( 'MS', closed='left', label='left' ).sum()
    if rR <= 1:
        # HRU 1
        H0_HR1_AET1 = H0_AET[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_HR1_AET1["R_%d" % rR] = H0_HR1_AET1[HRU_Labels[0]].cumsum( )
        H0_HR1_AET1.drop( columns=[HRU_Labels[0]], inplace=True)
        H0_HR1_AET2 = H0_AET[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_HR1_AET2["R_%d" % rR] = H0_HR1_AET2[HRU_Labels[0]].cumsum( )
        H0_HR1_AET2.drop( columns=[HRU_Labels[0]], inplace=True)
        H0_HR1_AET3 = H0_AET[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_HR1_AET3["R_%d" % rR] = H0_HR1_AET3[HRU_Labels[0]].cumsum( )
        H0_HR1_AET3.drop( columns=[HRU_Labels[0]], inplace=True)
        H0_HR1_AET4 = H0_AET[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_HR1_AET4["R_%d" % rR] = H0_HR1_AET4[HRU_Labels[0]].cumsum( )
        H0_HR1_AET4.drop( columns=[HRU_Labels[0]], inplace=True)
        H0_HR1_RO1 = H0_RO[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_HR1_RO1["R_%d" % rR] = H0_HR1_RO1[HRU_Labels[0]].cumsum( )
        H0_HR1_RO1.drop( columns=[HRU_Labels[0]], inplace=True)
        H0_HR1_RO2 = H0_RO[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_HR1_RO2["R_%d" % rR] = H0_HR1_RO2[HRU_Labels[0]].cumsum( )
        H0_HR1_RO2.drop( columns=[HRU_Labels[0]], inplace=True)
        H0_HR1_RO3 = H0_RO[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_HR1_RO3["R_%d" % rR] = H0_HR1_RO3[HRU_Labels[0]].cumsum( )
        H0_HR1_RO3.drop( columns=[HRU_Labels[0]], inplace=True)
        H0_HR1_RO4 = H0_RO[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_HR1_RO4["R_%d" % rR] = H0_HR1_RO4[HRU_Labels[0]].cumsum( )
        H0_HR1_RO4.drop( columns=[HRU_Labels[0]], inplace=True)
        H0_HR1_RE1 = H0_RE[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_HR1_RE1["R_%d" % rR] = H0_HR1_RE1[HRU_Labels[0]].cumsum( )
        H0_HR1_RE1.drop( columns=[HRU_Labels[0]], inplace=True)
        H0_HR1_RE2 = H0_RE[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_HR1_RE2["R_%d" % rR] = H0_HR1_RE2[HRU_Labels[0]].cumsum( )
        H0_HR1_RE2.drop( columns=[HRU_Labels[0]], inplace=True)
        H0_HR1_RE3 = H0_RE[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_HR1_RE3["R_%d" % rR] = H0_HR1_RE3[HRU_Labels[0]].cumsum( )
        H0_HR1_RE3.drop( columns=[HRU_Labels[0]], inplace=True)
        H0_HR1_RE4 = H0_RE[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_HR1_RE4["R_%d" % rR] = H0_HR1_RE4[HRU_Labels[0]].cumsum( )
        H0_HR1_RE4.drop( columns=[HRU_Labels[0]], inplace=True)
        # HRU 2
        H0_HR2_AET1 = H0_AET[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_HR2_AET1["R_%d" % rR] = H0_HR2_AET1[HRU_Labels[1]].cumsum( )
        H0_HR2_AET1.drop( columns=[HRU_Labels[1]], inplace=True)
        H0_HR2_AET2 = H0_AET[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_HR2_AET2["R_%d" % rR] = H0_HR2_AET2[HRU_Labels[1]].cumsum( )
        H0_HR2_AET2.drop( columns=[HRU_Labels[1]], inplace=True)
        H0_HR2_AET3 = H0_AET[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_HR2_AET3["R_%d" % rR] = H0_HR2_AET3[HRU_Labels[1]].cumsum( )
        H0_HR2_AET3.drop( columns=[HRU_Labels[1]], inplace=True)
        H0_HR2_AET4 = H0_AET[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_HR2_AET4["R_%d" % rR] = H0_HR2_AET4[HRU_Labels[1]].cumsum( )
        H0_HR2_AET4.drop( columns=[HRU_Labels[1]], inplace=True)
        H0_HR2_RO1 = H0_RO[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_HR2_RO1["R_%d" % rR] = H0_HR2_RO1[HRU_Labels[1]].cumsum( )
        H0_HR2_RO1.drop( columns=[HRU_Labels[1]], inplace=True)
        H0_HR2_RO2 = H0_RO[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_HR2_RO2["R_%d" % rR] = H0_HR2_RO2[HRU_Labels[1]].cumsum( )
        H0_HR2_RO2.drop( columns=[HRU_Labels[1]], inplace=True)
        H0_HR2_RO3 = H0_RO[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_HR2_RO3["R_%d" % rR] = H0_HR2_RO3[HRU_Labels[1]].cumsum( )
        H0_HR2_RO3.drop( columns=[HRU_Labels[1]], inplace=True)
        H0_HR2_RO4 = H0_RO[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_HR2_RO4["R_%d" % rR] = H0_HR2_RO4[HRU_Labels[1]].cumsum( )
        H0_HR2_RO4.drop( columns=[HRU_Labels[1]], inplace=True)
        H0_HR2_RE1 = H0_RE[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_HR2_RE1["R_%d" % rR] = H0_HR2_RE1[HRU_Labels[1]].cumsum( )
        H0_HR2_RE1.drop( columns=[HRU_Labels[1]], inplace=True)
        H0_HR2_RE2 = H0_RE[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_HR2_RE2["R_%d" % rR] = H0_HR2_RE2[HRU_Labels[1]].cumsum( )
        H0_HR2_RE2.drop( columns=[HRU_Labels[1]], inplace=True)
        H0_HR2_RE3 = H0_RE[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_HR2_RE3["R_%d" % rR] = H0_HR2_RE3[HRU_Labels[1]].cumsum( )
        H0_HR2_RE3.drop( columns=[HRU_Labels[1]], inplace=True)
        H0_HR2_RE4 = H0_RE[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_HR2_RE4["R_%d" % rR] = H0_HR2_RE4[HRU_Labels[1]].cumsum( )
        H0_HR2_RE4.drop( columns=[HRU_Labels[1]], inplace=True)
        # RR 1
        H0_RR1_AET1 = H0_AET[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR1_AET1["R_%d" % rR] = H0_RR1_AET1[RR_Labels[0]].cumsum( )
        H0_RR1_AET1.drop( columns=[RR_Labels[0]], inplace=True)
        H0_RR1_AET2 = H0_AET[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR1_AET2["R_%d" % rR] = H0_RR1_AET2[RR_Labels[0]].cumsum( )
        H0_RR1_AET2.drop( columns=[RR_Labels[0]], inplace=True)
        H0_RR1_AET3 = H0_AET[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR1_AET3["R_%d" % rR] = H0_RR1_AET3[RR_Labels[0]].cumsum( )
        H0_RR1_AET3.drop( columns=[RR_Labels[0]], inplace=True)
        H0_RR1_AET4 = H0_AET[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR1_AET4["R_%d" % rR] = H0_RR1_AET4[RR_Labels[0]].cumsum( )
        H0_RR1_AET4.drop( columns=[RR_Labels[0]], inplace=True)
        H0_RR1_RO1 = H0_RO[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR1_RO1["R_%d" % rR] = H0_RR1_RO1[RR_Labels[0]].cumsum( )
        H0_RR1_RO1.drop( columns=[RR_Labels[0]], inplace=True)
        H0_RR1_RO2 = H0_RO[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR1_RO2["R_%d" % rR] = H0_RR1_RO2[RR_Labels[0]].cumsum( )
        H0_RR1_RO2.drop( columns=[RR_Labels[0]], inplace=True)
        H0_RR1_RO3 = H0_RO[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR1_RO3["R_%d" % rR] = H0_RR1_RO3[RR_Labels[0]].cumsum( )
        H0_RR1_RO3.drop( columns=[RR_Labels[0]], inplace=True)
        H0_RR1_RO4 = H0_RO[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR1_RO4["R_%d" % rR] = H0_RR1_RO4[RR_Labels[0]].cumsum( )
        H0_RR1_RO4.drop( columns=[RR_Labels[0]], inplace=True)
        H0_RR1_RE1 = H0_RE[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR1_RE1["R_%d" % rR] = H0_RR1_RE1[RR_Labels[0]].cumsum( )
        H0_RR1_RE1.drop( columns=[RR_Labels[0]], inplace=True)
        H0_RR1_RE2 = H0_RE[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR1_RE2["R_%d" % rR] = H0_RR1_RE2[RR_Labels[0]].cumsum( )
        H0_RR1_RE2.drop( columns=[RR_Labels[0]], inplace=True)
        H0_RR1_RE3 = H0_RE[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR1_RE3["R_%d" % rR] = H0_RR1_RE3[RR_Labels[0]].cumsum( )
        H0_RR1_RE3.drop( columns=[RR_Labels[0]], inplace=True)
        H0_RR1_RE4 = H0_RE[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR1_RE4["R_%d" % rR] = H0_RR1_RE4[RR_Labels[0]].cumsum( )
        H0_RR1_RE4.drop( columns=[RR_Labels[0]], inplace=True)
        # RR 2
        H0_RR2_AET1 = H0_AET[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR2_AET1["R_%d" % rR] = H0_RR2_AET1[RR_Labels[1]].cumsum( )
        H0_RR2_AET1.drop( columns=[RR_Labels[1]], inplace=True)
        H0_RR2_AET2 = H0_AET[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR2_AET2["R_%d" % rR] = H0_RR2_AET2[RR_Labels[1]].cumsum( )
        H0_RR2_AET2.drop( columns=[RR_Labels[1]], inplace=True)
        H0_RR2_AET3 = H0_AET[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR2_AET3["R_%d" % rR] = H0_RR2_AET3[RR_Labels[1]].cumsum( )
        H0_RR2_AET3.drop( columns=[RR_Labels[1]], inplace=True)
        H0_RR2_AET4 = H0_AET[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR2_AET4["R_%d" % rR] = H0_RR2_AET4[RR_Labels[1]].cumsum( )
        H0_RR2_AET4.drop( columns=[RR_Labels[1]], inplace=True)
        H0_RR2_RO1 = H0_RO[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR2_RO1["R_%d" % rR] = H0_RR2_RO1[RR_Labels[1]].cumsum( )
        H0_RR2_RO1.drop( columns=[RR_Labels[1]], inplace=True)
        H0_RR2_RO2 = H0_RO[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR2_RO2["R_%d" % rR] = H0_RR2_RO2[RR_Labels[1]].cumsum( )
        H0_RR2_RO2.drop( columns=[RR_Labels[1]], inplace=True)
        H0_RR2_RO3 = H0_RO[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR2_RO3["R_%d" % rR] = H0_RR2_RO3[RR_Labels[1]].cumsum( )
        H0_RR2_RO3.drop( columns=[RR_Labels[1]], inplace=True)
        H0_RR2_RO4 = H0_RO[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR2_RO4["R_%d" % rR] = H0_RR2_RO4[RR_Labels[1]].cumsum( )
        H0_RR2_RO4.drop( columns=[RR_Labels[1]], inplace=True)
        H0_RR2_RE1 = H0_RE[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR2_RE1["R_%d" % rR] = H0_RR2_RE1[RR_Labels[1]].cumsum( )
        H0_RR2_RE1.drop( columns=[RR_Labels[1]], inplace=True)
        H0_RR2_RE2 = H0_RE[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR2_RE2["R_%d" % rR] = H0_RR2_RE2[RR_Labels[1]].cumsum( )
        H0_RR2_RE2.drop( columns=[RR_Labels[1]], inplace=True)
        H0_RR2_RE3 = H0_RE[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR2_RE3["R_%d" % rR] = H0_RR2_RE3[RR_Labels[1]].cumsum( )
        H0_RR2_RE3.drop( columns=[RR_Labels[1]], inplace=True)
        H0_RR2_RE4 = H0_RE[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR2_RE4["R_%d" % rR] = H0_RR2_RE4[RR_Labels[1]].cumsum( )
        H0_RR2_RE4.drop( columns=[RR_Labels[1]], inplace=True)
        # RR 3
        H0_RR3_AET1 = H0_AET[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR3_AET1["R_%d" % rR] = H0_RR3_AET1[RR_Labels[2]].cumsum( )
        H0_RR3_AET1.drop( columns=[RR_Labels[2]], inplace=True)
        H0_RR3_AET2 = H0_AET[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR3_AET2["R_%d" % rR] = H0_RR3_AET2[RR_Labels[2]].cumsum( )
        H0_RR3_AET2.drop( columns=[RR_Labels[2]], inplace=True)
        H0_RR3_AET3 = H0_AET[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR3_AET3["R_%d" % rR] = H0_RR3_AET3[RR_Labels[2]].cumsum( )
        H0_RR3_AET3.drop( columns=[RR_Labels[2]], inplace=True)
        H0_RR3_AET4 = H0_AET[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR3_AET4["R_%d" % rR] = H0_RR3_AET4[RR_Labels[2]].cumsum( )
        H0_RR3_AET4.drop( columns=[RR_Labels[2]], inplace=True)
        H0_RR3_RO1 = H0_RO[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR3_RO1["R_%d" % rR] = H0_RR3_RO1[RR_Labels[2]].cumsum( )
        H0_RR3_RO1.drop( columns=[RR_Labels[2]], inplace=True)
        H0_RR3_RO2 = H0_RO[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR3_RO2["R_%d" % rR] = H0_RR3_RO2[RR_Labels[2]].cumsum( )
        H0_RR3_RO2.drop( columns=[RR_Labels[2]], inplace=True)
        H0_RR3_RO3 = H0_RO[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR3_RO3["R_%d" % rR] = H0_RR3_RO3[RR_Labels[2]].cumsum( )
        H0_RR3_RO3.drop( columns=[RR_Labels[2]], inplace=True)
        H0_RR3_RO4 = H0_RO[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR3_RO4["R_%d" % rR] = H0_RR3_RO4[RR_Labels[2]].cumsum( )
        H0_RR3_RO4.drop( columns=[RR_Labels[2]], inplace=True)
        H0_RR3_RE1 = H0_RE[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR3_RE1["R_%d" % rR] = H0_RR3_RE1[RR_Labels[2]].cumsum( )
        H0_RR3_RE1.drop( columns=[RR_Labels[2]], inplace=True)
        H0_RR3_RE2 = H0_RE[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR3_RE2["R_%d" % rR] = H0_RR3_RE2[RR_Labels[2]].cumsum( )
        H0_RR3_RE2.drop( columns=[RR_Labels[2]], inplace=True)
        H0_RR3_RE3 = H0_RE[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR3_RE3["R_%d" % rR] = H0_RR3_RE3[RR_Labels[2]].cumsum( )
        H0_RR3_RE3.drop( columns=[RR_Labels[2]], inplace=True)
        H0_RR3_RE4 = H0_RE[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR3_RE4["R_%d" % rR] = H0_RR3_RE4[RR_Labels[2]].cumsum( )
        H0_RR3_RE4.drop( columns=[RR_Labels[2]], inplace=True)
        # RR 4
        H0_RR4_AET1 = H0_AET[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR4_AET1["R_%d" % rR] = H0_RR4_AET1[RR_Labels[3]].cumsum( )
        H0_RR4_AET1.drop( columns=[RR_Labels[3]], inplace=True)
        H0_RR4_AET2 = H0_AET[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR4_AET2["R_%d" % rR] = H0_RR4_AET2[RR_Labels[3]].cumsum( )
        H0_RR4_AET2.drop( columns=[RR_Labels[3]], inplace=True)
        H0_RR4_AET3 = H0_AET[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR4_AET3["R_%d" % rR] = H0_RR4_AET3[RR_Labels[3]].cumsum( )
        H0_RR4_AET3.drop( columns=[RR_Labels[3]], inplace=True)
        H0_RR4_AET4 = H0_AET[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR4_AET4["R_%d" % rR] = H0_RR4_AET4[RR_Labels[3]].cumsum( )
        H0_RR4_AET4.drop( columns=[RR_Labels[3]], inplace=True)
        H0_RR4_RO1 = H0_RO[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR4_RO1["R_%d" % rR] = H0_RR4_RO1[RR_Labels[3]].cumsum( )
        H0_RR4_RO1.drop( columns=[RR_Labels[3]], inplace=True)
        H0_RR4_RO2 = H0_RO[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR4_RO2["R_%d" % rR] = H0_RR4_RO2[RR_Labels[3]].cumsum( )
        H0_RR4_RO2.drop( columns=[RR_Labels[3]], inplace=True)
        H0_RR4_RO3 = H0_RO[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR4_RO3["R_%d" % rR] = H0_RR4_RO3[RR_Labels[3]].cumsum( )
        H0_RR4_RO3.drop( columns=[RR_Labels[3]], inplace=True)
        H0_RR4_RO4 = H0_RO[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR4_RO4["R_%d" % rR] = H0_RR4_RO4[RR_Labels[3]].cumsum( )
        H0_RR4_RO4.drop( columns=[RR_Labels[3]], inplace=True)
        H0_RR4_RE1 = H0_RE[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR4_RE1["R_%d" % rR] = H0_RR4_RE1[RR_Labels[3]].cumsum( )
        H0_RR4_RE1.drop( columns=[RR_Labels[3]], inplace=True)
        H0_RR4_RE2 = H0_RE[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR4_RE2["R_%d" % rR] = H0_RR4_RE2[RR_Labels[3]].cumsum( )
        H0_RR4_RE2.drop( columns=[RR_Labels[3]], inplace=True)
        H0_RR4_RE3 = H0_RE[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR4_RE3["R_%d" % rR] = H0_RR4_RE3[RR_Labels[3]].cumsum( )
        H0_RR4_RE3.drop( columns=[RR_Labels[3]], inplace=True)
        H0_RR4_RE4 = H0_RE[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR4_RE4["R_%d" % rR] = H0_RR4_RE4[RR_Labels[3]].cumsum( )
        H0_RR4_RE4.drop( columns=[RR_Labels[3]], inplace=True)
        # RR 5
        H0_RR5_AET1 = H0_AET[[RR_Labels[4]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR5_AET1["R_%d" % rR] = H0_RR5_AET1[RR_Labels[4]].cumsum( )
        H0_RR5_AET1.drop( columns=[RR_Labels[4]], inplace=True)
        H0_RR5_AET2 = H0_AET[[RR_Labels[4]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR5_AET2["R_%d" % rR] = H0_RR5_AET2[RR_Labels[4]].cumsum( )
        H0_RR5_AET2.drop( columns=[RR_Labels[4]], inplace=True)
        H0_RR5_AET3 = H0_AET[[RR_Labels[4]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR5_AET3["R_%d" % rR] = H0_RR5_AET3[RR_Labels[4]].cumsum( )
        H0_RR5_AET3.drop( columns=[RR_Labels[4]], inplace=True)
        H0_RR5_AET4 = H0_AET[[RR_Labels[4]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR5_AET4["R_%d" % rR] = H0_RR5_AET4[RR_Labels[4]].cumsum( )
        H0_RR5_AET4.drop( columns=[RR_Labels[4]], inplace=True)
        H0_RR5_RO1 = H0_RO[[RR_Labels[4]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H0_RR5_RO1["R_%d" % rR] = H0_RR5_RO1[RR_Labels[4]].cumsum( )
        H0_RR5_RO1.drop( columns=[RR_Labels[4]], inplace=True)
        H0_RR5_RO2 = H0_RO[[RR_Labels[4]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H0_RR5_RO2["R_%d" % rR] = H0_RR5_RO2[RR_Labels[4]].cumsum( )
        H0_RR5_RO2.drop( columns=[RR_Labels[4]], inplace=True)
        H0_RR5_RO3 = H0_RO[[RR_Labels[4]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H0_RR5_RO3["R_%d" % rR] = H0_RR5_RO3[RR_Labels[4]].cumsum( )
        H0_RR5_RO3.drop( columns=[RR_Labels[4]], inplace=True)
        H0_RR5_RO4 = H0_RO[[RR_Labels[4]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H0_RR5_RO4["R_%d" % rR] = H0_RR5_RO4[RR_Labels[4]].cumsum( )
        H0_RR5_RO4.drop( columns=[RR_Labels[4]], inplace=True)
    else:
        # HRU 1
        tmpAET = H0_AET[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[0]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[0]], inplace=True )
        H0_HR1_AET1 = H0_HR1_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[0]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[0]], inplace=True )
        H0_HR1_AET2 = H0_HR1_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[0]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[0]], inplace=True )
        H0_HR1_AET3 = H0_HR1_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[0]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[0]], inplace=True )
        H0_HR1_AET4 = H0_HR1_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[0]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[0]], inplace=True )
        H0_HR1_RO1 = H0_HR1_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[0]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[0]], inplace=True )
        H0_HR1_RO2 = H0_HR1_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[0]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[0]], inplace=True )
        H0_HR1_RO3 = H0_HR1_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[0]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[0]], inplace=True )
        H0_HR1_RO4 = H0_HR1_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[0]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[0]], inplace=True )
        H0_HR1_RE1 = H0_HR1_RE1.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[0]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[0]], inplace=True )
        H0_HR1_RE2 = H0_HR1_RE2.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[0]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[0]], inplace=True )
        H0_HR1_RE3 = H0_HR1_RE3.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[0]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[0]], inplace=True )
        H0_HR1_RE4 = H0_HR1_RE4.merge( tmpRE, how='inner', left_index=True, right_index=True )
        # HRU 2
        tmpAET = H0_AET[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[1]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[1]], inplace=True )
        H0_HR2_AET1 = H0_HR2_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[1]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[1]], inplace=True )
        H0_HR2_AET2 = H0_HR2_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[1]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[1]], inplace=True )
        H0_HR2_AET3 = H0_HR2_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[1]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[1]], inplace=True )
        H0_HR2_AET4 = H0_HR2_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[1]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[1]], inplace=True )
        H0_HR2_RO1 = H0_HR2_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[1]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[1]], inplace=True )
        H0_HR2_RO2 = H0_HR2_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[1]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[1]], inplace=True )
        H0_HR2_RO3 = H0_HR2_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[1]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[1]], inplace=True )
        H0_HR2_RO4 = H0_HR2_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[1]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[1]], inplace=True )
        H0_HR2_RE1 = H0_HR2_RE1.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[1]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[1]], inplace=True )
        H0_HR2_RE2 = H0_HR2_RE2.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[1]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[1]], inplace=True )
        H0_HR2_RE3 = H0_HR2_RE3.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[1]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[1]], inplace=True )
        H0_HR2_RE4 = H0_HR2_RE4.merge( tmpRE, how='inner', left_index=True, right_index=True )
        # RR 1
        tmpAET = H0_AET[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[0]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[0]], inplace=True )
        H0_RR1_AET1 = H0_RR1_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[0]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[0]], inplace=True )
        H0_RR1_AET2 = H0_RR1_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[0]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[0]], inplace=True )
        H0_RR1_AET3 = H0_RR1_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[0]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[0]], inplace=True )
        H0_RR1_AET4 = H0_RR1_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[0]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[0]], inplace=True )
        H0_RR1_RO1 = H0_RR1_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[0]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[0]], inplace=True )
        H0_RR1_RO2 = H0_RR1_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[0]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[0]], inplace=True )
        H0_RR1_RO3 = H0_RR1_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[0]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[0]], inplace=True )
        H0_RR1_RO4 = H0_RR1_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[0]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[0]], inplace=True )
        H0_RR1_RE1 = H0_RR1_RE1.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[0]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[0]], inplace=True )
        H0_RR1_RE2 = H0_RR1_RE2.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[0]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[0]], inplace=True )
        H0_RR1_RE3 = H0_RR1_RE3.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[0]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[0]], inplace=True )
        H0_RR1_RE4 = H0_RR1_RE4.merge( tmpRE, how='inner', left_index=True, right_index=True )
        # RR 2
        tmpAET = H0_AET[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[1]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[1]], inplace=True )
        H0_RR2_AET1 = H0_RR2_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[1]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[1]], inplace=True )
        H0_RR2_AET2 = H0_RR2_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[1]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[1]], inplace=True )
        H0_RR2_AET3 = H0_RR2_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[1]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[1]], inplace=True )
        H0_RR2_AET4 = H0_RR2_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[1]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[1]], inplace=True )
        H0_RR2_RO1 = H0_RR2_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[1]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[1]], inplace=True )
        H0_RR2_RO2 = H0_RR2_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[1]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[1]], inplace=True )
        H0_RR2_RO3 = H0_RR2_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[1]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[1]], inplace=True )
        H0_RR2_RO4 = H0_RR2_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[1]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[1]], inplace=True )
        H0_RR2_RE1 = H0_RR2_RE1.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[1]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[1]], inplace=True )
        H0_RR2_RE2 = H0_RR2_RE2.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[1]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[1]], inplace=True )
        H0_RR2_RE3 = H0_RR2_RE3.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[1]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[1]], inplace=True )
        H0_RR2_RE4 = H0_RR2_RE4.merge( tmpRE, how='inner', left_index=True, right_index=True )
        # RR 3
        tmpAET = H0_AET[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[2]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[2]], inplace=True )
        H0_RR3_AET1 = H0_RR3_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[2]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[2]], inplace=True )
        H0_RR3_AET2 = H0_RR3_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[2]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[2]], inplace=True )
        H0_RR3_AET3 = H0_RR3_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[2]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[2]], inplace=True )
        H0_RR3_AET4 = H0_RR3_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[2]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[2]], inplace=True )
        H0_RR3_RO1 = H0_RR3_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[2]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[2]], inplace=True )
        H0_RR3_RO2 = H0_RR3_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[2]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[2]], inplace=True )
        H0_RR3_RO3 = H0_RR3_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[2]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[2]], inplace=True )
        H0_RR3_RO4 = H0_RR3_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[2]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[2]], inplace=True )
        H0_RR3_RE1 = H0_RR3_RE1.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[2]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[2]], inplace=True )
        H0_RR3_RE2 = H0_RR3_RE2.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[2]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[2]], inplace=True )
        H0_RR3_RE3 = H0_RR3_RE3.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[2]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[2]], inplace=True )
        H0_RR3_RE4 = H0_RR3_RE4.merge( tmpRE, how='inner', left_index=True, right_index=True )
        # RR 4
        tmpAET = H0_AET[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[3]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[3]], inplace=True )
        H0_RR4_AET1 = H0_RR4_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[3]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[3]], inplace=True )
        H0_RR4_AET2 = H0_RR4_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[3]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[3]], inplace=True )
        H0_RR4_AET3 = H0_RR4_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[3]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[3]], inplace=True )
        H0_RR4_AET4 = H0_RR4_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[3]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[3]], inplace=True )
        H0_RR4_RO1 = H0_RR4_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[3]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[3]], inplace=True )
        H0_RR4_RO2 = H0_RR4_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[3]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[3]], inplace=True )
        H0_RR4_RO3 = H0_RR4_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[3]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[3]], inplace=True )
        H0_RR4_RO4 = H0_RR4_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[3]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[3]], inplace=True )
        H0_RR4_RE1 = H0_RR4_RE1.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[3]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[3]], inplace=True )
        H0_RR4_RE2 = H0_RR4_RE2.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[3]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[3]], inplace=True )
        H0_RR4_RE3 = H0_RR4_RE3.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H0_RE[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[3]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[3]], inplace=True )
        H0_RR4_RE4 = H0_RR4_RE4.merge( tmpRE, how='inner', left_index=True, right_index=True )
        # RR 5
        tmpAET = H0_AET[[RR_Labels[4]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[4]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[4]], inplace=True )
        H0_RR5_AET1 = H0_RR5_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[4]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[4]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[4]], inplace=True )
        H0_RR5_AET2 = H0_RR5_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[4]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[4]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[4]], inplace=True )
        H0_RR5_AET3 = H0_RR5_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H0_AET[[RR_Labels[4]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[4]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[4]], inplace=True )
        H0_RR5_AET4 = H0_RR5_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[4]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[4]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[4]], inplace=True )
        H0_RR5_RO1 = H0_RR5_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[4]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[4]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[4]], inplace=True )
        H0_RR5_RO2 = H0_RR5_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[4]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[4]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[4]], inplace=True )
        H0_RR5_RO3 = H0_RR5_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H0_RO[[RR_Labels[4]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[4]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[4]], inplace=True )
        H0_RR5_RO4 = H0_RR5_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )       
    # end if
# end for

Working on realization 200
Working on realization 400
Working on realization 600
Working on realization 800
Working on realization 1000
Working on realization 1200
Working on realization 1400
Working on realization 1600
Working on realization 1800
Working on realization 2000
Working on realization 2200
Working on realization 2400
Working on realization 2600
Working on realization 2800
Working on realization 3000
Working on realization 3200
Working on realization 3400
Working on realization 3600
Working on realization 3800
Working on realization 4000
Working on realization 4200
Working on realization 4400
Working on realization 4600
Working on realization 4800
Working on realization 5000
Working on realization 5200
Working on realization 5400
Working on realization 5600
Working on realization 5800
Working on realization 6000
Working on realization 6200
Working on realization 6400
Working on realization 6600
Working on realization 6800
Working on realization 7000
Working on realization 7

This loop takes a long time so output this files to pickles now

In [12]:
OFNamer = "Cum_H0_HRU1_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR1_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU1_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR1_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU1_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR1_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU1_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR1_AET4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU2_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR2_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU2_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR2_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU2_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR2_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU2_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR2_AET4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR1_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR1_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR1_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR1_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR1_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR1_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR1_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR1_AET4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR2_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR2_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR2_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR2_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR2_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR2_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR2_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR2_AET4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR3_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR3_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR3_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR3_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR3_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR3_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR3_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR3_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR4_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR4_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR4_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR4_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR4_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR4_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR4_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR4_AET4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR5_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR5_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR5_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR5_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR5_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR5_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR5_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR5_AET4.to_pickle( OFPath, compression='zip' )

In [13]:
OFNamer = "Cum_H0_HRU1_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR1_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU1_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR1_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU1_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR1_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU1_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR1_RO4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU2_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR2_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU2_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR2_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU2_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR2_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU2_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR2_RO4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR1_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR1_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR1_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR1_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR1_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR1_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR1_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR1_RO4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR2_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR2_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR2_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR2_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR2_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR2_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR2_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR2_RO4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR3_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR3_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR3_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR3_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR3_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR3_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR3_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR3_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR4_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR4_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR4_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR4_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR4_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR4_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR4_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR4_RO4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR5_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR5_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR5_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR5_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR5_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR5_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR5_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR5_RO4.to_pickle( OFPath, compression='zip' )

In [14]:
OFNamer = "Cum_H0_HRU1_RE1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR1_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU1_RE2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR1_RE2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU1_RE3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR1_RE3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU1_RE4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR1_RE4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU2_RE1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR2_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU2_RE2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR2_RE2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU2_RE3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR2_RE3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_HRU2_RE4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_HR2_RE4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR1_RE1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR1_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR1_RE2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR1_RE2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR1_RE3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR1_RE3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR1_RE4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR1_RE4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR2_RE1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR2_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR2_RE2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR2_RE2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR2_RE3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR2_RE3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR2_RE4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR2_RE4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR3_RE1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR3_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR3_RE2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR3_RE2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR3_RE3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR3_RE3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR3_RE4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR3_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR4_RE1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR4_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR4_RE2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR4_RE2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR4_RE3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR4_RE3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H0_RR4_RE4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H0_RR4_RE4.to_pickle( OFPath, compression='zip' )

### H1

Go through all realizations and create a DataFrame for each constituent and each period

In [15]:
for rR in range( 1, NUMREAL + 1, 1 ):
    if ((rR % 200) == 0):
        print("Working on realization %d" % rR)
    # end if
    # AET
    InFile = IN_RAET1 % rR
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    InDF = InDF[AET_In_Hdrs].copy()
    H1_AET = InDF.resample( 'MS', closed='left', label='left' ).sum()
    # RO
    InFile = IN_RRO1 % rR
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    InDF = InDF[RO_In_Hdrs].copy()
    H1_RO = InDF.resample( 'MS', closed='left', label='left' ).sum()
    # Re
    InFile = IN_RRE1 % rR
    InFP = os.path.normpath( os.path.join( IN_DIR, InFile ) )
    InDF = pd.read_pickle( InFP, compression='zip' )
    InDF = InDF[RE_In_Hdrs].copy()
    H1_RE = InDF.resample( 'MS', closed='left', label='left' ).sum()
    if rR <= 1:
        # HRU 1
        H1_HR1_AET1 = H1_AET[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_HR1_AET1["R_%d" % rR] = H1_HR1_AET1[HRU_Labels[0]].cumsum( )
        H1_HR1_AET1.drop( columns=[HRU_Labels[0]], inplace=True)
        H1_HR1_AET2 = H1_AET[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_HR1_AET2["R_%d" % rR] = H1_HR1_AET2[HRU_Labels[0]].cumsum( )
        H1_HR1_AET2.drop( columns=[HRU_Labels[0]], inplace=True)
        H1_HR1_AET3 = H1_AET[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_HR1_AET3["R_%d" % rR] = H1_HR1_AET3[HRU_Labels[0]].cumsum( )
        H1_HR1_AET3.drop( columns=[HRU_Labels[0]], inplace=True)
        H1_HR1_AET4 = H1_AET[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_HR1_AET4["R_%d" % rR] = H1_HR1_AET4[HRU_Labels[0]].cumsum( )
        H1_HR1_AET4.drop( columns=[HRU_Labels[0]], inplace=True)
        H1_HR1_RO1 = H1_RO[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_HR1_RO1["R_%d" % rR] = H1_HR1_RO1[HRU_Labels[0]].cumsum( )
        H1_HR1_RO1.drop( columns=[HRU_Labels[0]], inplace=True)
        H1_HR1_RO2 = H1_RO[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_HR1_RO2["R_%d" % rR] = H1_HR1_RO2[HRU_Labels[0]].cumsum( )
        H1_HR1_RO2.drop( columns=[HRU_Labels[0]], inplace=True)
        H1_HR1_RO3 = H1_RO[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_HR1_RO3["R_%d" % rR] = H1_HR1_RO3[HRU_Labels[0]].cumsum( )
        H1_HR1_RO3.drop( columns=[HRU_Labels[0]], inplace=True)
        H1_HR1_RO4 = H1_RO[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_HR1_RO4["R_%d" % rR] = H1_HR1_RO4[HRU_Labels[0]].cumsum( )
        H1_HR1_RO4.drop( columns=[HRU_Labels[0]], inplace=True)
        H1_HR1_RE1 = H1_RE[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_HR1_RE1["R_%d" % rR] = H1_HR1_RE1[HRU_Labels[0]].cumsum( )
        H1_HR1_RE1.drop( columns=[HRU_Labels[0]], inplace=True)
        H1_HR1_RE2 = H1_RE[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_HR1_RE2["R_%d" % rR] = H1_HR1_RE2[HRU_Labels[0]].cumsum( )
        H1_HR1_RE2.drop( columns=[HRU_Labels[0]], inplace=True)
        H1_HR1_RE3 = H1_RE[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_HR1_RE3["R_%d" % rR] = H1_HR1_RE3[HRU_Labels[0]].cumsum( )
        H1_HR1_RE3.drop( columns=[HRU_Labels[0]], inplace=True)
        H1_HR1_RE4 = H1_RE[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_HR1_RE4["R_%d" % rR] = H1_HR1_RE4[HRU_Labels[0]].cumsum( )
        H1_HR1_RE4.drop( columns=[HRU_Labels[0]], inplace=True)
        # HRU 2
        H1_HR2_AET1 = H1_AET[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_HR2_AET1["R_%d" % rR] = H1_HR2_AET1[HRU_Labels[1]].cumsum( )
        H1_HR2_AET1.drop( columns=[HRU_Labels[1]], inplace=True)
        H1_HR2_AET2 = H1_AET[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_HR2_AET2["R_%d" % rR] = H1_HR2_AET2[HRU_Labels[1]].cumsum( )
        H1_HR2_AET2.drop( columns=[HRU_Labels[1]], inplace=True)
        H1_HR2_AET3 = H1_AET[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_HR2_AET3["R_%d" % rR] = H1_HR2_AET3[HRU_Labels[1]].cumsum( )
        H1_HR2_AET3.drop( columns=[HRU_Labels[1]], inplace=True)
        H1_HR2_AET4 = H1_AET[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_HR2_AET4["R_%d" % rR] = H1_HR2_AET4[HRU_Labels[1]].cumsum( )
        H1_HR2_AET4.drop( columns=[HRU_Labels[1]], inplace=True)
        H1_HR2_RO1 = H1_RO[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_HR2_RO1["R_%d" % rR] = H1_HR2_RO1[HRU_Labels[1]].cumsum( )
        H1_HR2_RO1.drop( columns=[HRU_Labels[1]], inplace=True)
        H1_HR2_RO2 = H1_RO[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_HR2_RO2["R_%d" % rR] = H1_HR2_RO2[HRU_Labels[1]].cumsum( )
        H1_HR2_RO2.drop( columns=[HRU_Labels[1]], inplace=True)
        H1_HR2_RO3 = H1_RO[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_HR2_RO3["R_%d" % rR] = H1_HR2_RO3[HRU_Labels[1]].cumsum( )
        H1_HR2_RO3.drop( columns=[HRU_Labels[1]], inplace=True)
        H1_HR2_RO4 = H1_RO[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_HR2_RO4["R_%d" % rR] = H1_HR2_RO4[HRU_Labels[1]].cumsum( )
        H1_HR2_RO4.drop( columns=[HRU_Labels[1]], inplace=True)
        H1_HR2_RE1 = H1_RE[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_HR2_RE1["R_%d" % rR] = H1_HR2_RE1[HRU_Labels[1]].cumsum( )
        H1_HR2_RE1.drop( columns=[HRU_Labels[1]], inplace=True)
        H1_HR2_RE2 = H1_RE[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_HR2_RE2["R_%d" % rR] = H1_HR2_RE2[HRU_Labels[1]].cumsum( )
        H1_HR2_RE2.drop( columns=[HRU_Labels[1]], inplace=True)
        H1_HR2_RE3 = H1_RE[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_HR2_RE3["R_%d" % rR] = H1_HR2_RE3[HRU_Labels[1]].cumsum( )
        H1_HR2_RE3.drop( columns=[HRU_Labels[1]], inplace=True)
        H1_HR2_RE4 = H1_RE[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_HR2_RE4["R_%d" % rR] = H1_HR2_RE4[HRU_Labels[1]].cumsum( )
        H1_HR2_RE4.drop( columns=[HRU_Labels[1]], inplace=True)
        # RR 1
        H1_RR1_AET1 = H1_AET[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR1_AET1["R_%d" % rR] = H1_RR1_AET1[RR_Labels[0]].cumsum( )
        H1_RR1_AET1.drop( columns=[RR_Labels[0]], inplace=True)
        H1_RR1_AET2 = H1_AET[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR1_AET2["R_%d" % rR] = H1_RR1_AET2[RR_Labels[0]].cumsum( )
        H1_RR1_AET2.drop( columns=[RR_Labels[0]], inplace=True)
        H1_RR1_AET3 = H1_AET[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR1_AET3["R_%d" % rR] = H1_RR1_AET3[RR_Labels[0]].cumsum( )
        H1_RR1_AET3.drop( columns=[RR_Labels[0]], inplace=True)
        H1_RR1_AET4 = H1_AET[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR1_AET4["R_%d" % rR] = H1_RR1_AET4[RR_Labels[0]].cumsum( )
        H1_RR1_AET4.drop( columns=[RR_Labels[0]], inplace=True)
        H1_RR1_RO1 = H1_RO[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR1_RO1["R_%d" % rR] = H1_RR1_RO1[RR_Labels[0]].cumsum( )
        H1_RR1_RO1.drop( columns=[RR_Labels[0]], inplace=True)
        H1_RR1_RO2 = H1_RO[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR1_RO2["R_%d" % rR] = H1_RR1_RO2[RR_Labels[0]].cumsum( )
        H1_RR1_RO2.drop( columns=[RR_Labels[0]], inplace=True)
        H1_RR1_RO3 = H1_RO[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR1_RO3["R_%d" % rR] = H1_RR1_RO3[RR_Labels[0]].cumsum( )
        H1_RR1_RO3.drop( columns=[RR_Labels[0]], inplace=True)
        H1_RR1_RO4 = H1_RO[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR1_RO4["R_%d" % rR] = H1_RR1_RO4[RR_Labels[0]].cumsum( )
        H1_RR1_RO4.drop( columns=[RR_Labels[0]], inplace=True)
        H1_RR1_RE1 = H1_RE[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR1_RE1["R_%d" % rR] = H1_RR1_RE1[RR_Labels[0]].cumsum( )
        H1_RR1_RE1.drop( columns=[RR_Labels[0]], inplace=True)
        H1_RR1_RE2 = H1_RE[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR1_RE2["R_%d" % rR] = H1_RR1_RE2[RR_Labels[0]].cumsum( )
        H1_RR1_RE2.drop( columns=[RR_Labels[0]], inplace=True)
        H1_RR1_RE3 = H1_RE[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR1_RE3["R_%d" % rR] = H1_RR1_RE3[RR_Labels[0]].cumsum( )
        H1_RR1_RE3.drop( columns=[RR_Labels[0]], inplace=True)
        H1_RR1_RE4 = H1_RE[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR1_RE4["R_%d" % rR] = H1_RR1_RE4[RR_Labels[0]].cumsum( )
        H1_RR1_RE4.drop( columns=[RR_Labels[0]], inplace=True)
        # RR 2
        H1_RR2_AET1 = H1_AET[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR2_AET1["R_%d" % rR] = H1_RR2_AET1[RR_Labels[1]].cumsum( )
        H1_RR2_AET1.drop( columns=[RR_Labels[1]], inplace=True)
        H1_RR2_AET2 = H1_AET[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR2_AET2["R_%d" % rR] = H1_RR2_AET2[RR_Labels[1]].cumsum( )
        H1_RR2_AET2.drop( columns=[RR_Labels[1]], inplace=True)
        H1_RR2_AET3 = H1_AET[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR2_AET3["R_%d" % rR] = H1_RR2_AET3[RR_Labels[1]].cumsum( )
        H1_RR2_AET3.drop( columns=[RR_Labels[1]], inplace=True)
        H1_RR2_AET4 = H1_AET[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR2_AET4["R_%d" % rR] = H1_RR2_AET4[RR_Labels[1]].cumsum( )
        H1_RR2_AET4.drop( columns=[RR_Labels[1]], inplace=True)
        H1_RR2_RO1 = H1_RO[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR2_RO1["R_%d" % rR] = H1_RR2_RO1[RR_Labels[1]].cumsum( )
        H1_RR2_RO1.drop( columns=[RR_Labels[1]], inplace=True)
        H1_RR2_RO2 = H1_RO[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR2_RO2["R_%d" % rR] = H1_RR2_RO2[RR_Labels[1]].cumsum( )
        H1_RR2_RO2.drop( columns=[RR_Labels[1]], inplace=True)
        H1_RR2_RO3 = H1_RO[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR2_RO3["R_%d" % rR] = H1_RR2_RO3[RR_Labels[1]].cumsum( )
        H1_RR2_RO3.drop( columns=[RR_Labels[1]], inplace=True)
        H1_RR2_RO4 = H1_RO[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR2_RO4["R_%d" % rR] = H1_RR2_RO4[RR_Labels[1]].cumsum( )
        H1_RR2_RO4.drop( columns=[RR_Labels[1]], inplace=True)
        H1_RR2_RE1 = H1_RE[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR2_RE1["R_%d" % rR] = H1_RR2_RE1[RR_Labels[1]].cumsum( )
        H1_RR2_RE1.drop( columns=[RR_Labels[1]], inplace=True)
        H1_RR2_RE2 = H1_RE[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR2_RE2["R_%d" % rR] = H1_RR2_RE2[RR_Labels[1]].cumsum( )
        H1_RR2_RE2.drop( columns=[RR_Labels[1]], inplace=True)
        H1_RR2_RE3 = H1_RE[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR2_RE3["R_%d" % rR] = H1_RR2_RE3[RR_Labels[1]].cumsum( )
        H1_RR2_RE3.drop( columns=[RR_Labels[1]], inplace=True)
        H1_RR2_RE4 = H1_RE[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR2_RE4["R_%d" % rR] = H1_RR2_RE4[RR_Labels[1]].cumsum( )
        H1_RR2_RE4.drop( columns=[RR_Labels[1]], inplace=True)
        # RR 3
        H1_RR3_AET1 = H1_AET[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR3_AET1["R_%d" % rR] = H1_RR3_AET1[RR_Labels[2]].cumsum( )
        H1_RR3_AET1.drop( columns=[RR_Labels[2]], inplace=True)
        H1_RR3_AET2 = H1_AET[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR3_AET2["R_%d" % rR] = H1_RR3_AET2[RR_Labels[2]].cumsum( )
        H1_RR3_AET2.drop( columns=[RR_Labels[2]], inplace=True)
        H1_RR3_AET3 = H1_AET[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR3_AET3["R_%d" % rR] = H1_RR3_AET3[RR_Labels[2]].cumsum( )
        H1_RR3_AET3.drop( columns=[RR_Labels[2]], inplace=True)
        H1_RR3_AET4 = H1_AET[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR3_AET4["R_%d" % rR] = H1_RR3_AET4[RR_Labels[2]].cumsum( )
        H1_RR3_AET4.drop( columns=[RR_Labels[2]], inplace=True)
        H1_RR3_RO1 = H1_RO[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR3_RO1["R_%d" % rR] = H1_RR3_RO1[RR_Labels[2]].cumsum( )
        H1_RR3_RO1.drop( columns=[RR_Labels[2]], inplace=True)
        H1_RR3_RO2 = H1_RO[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR3_RO2["R_%d" % rR] = H1_RR3_RO2[RR_Labels[2]].cumsum( )
        H1_RR3_RO2.drop( columns=[RR_Labels[2]], inplace=True)
        H1_RR3_RO3 = H1_RO[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR3_RO3["R_%d" % rR] = H1_RR3_RO3[RR_Labels[2]].cumsum( )
        H1_RR3_RO3.drop( columns=[RR_Labels[2]], inplace=True)
        H1_RR3_RO4 = H1_RO[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR3_RO4["R_%d" % rR] = H1_RR3_RO4[RR_Labels[2]].cumsum( )
        H1_RR3_RO4.drop( columns=[RR_Labels[2]], inplace=True)
        H1_RR3_RE1 = H1_RE[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR3_RE1["R_%d" % rR] = H1_RR3_RE1[RR_Labels[2]].cumsum( )
        H1_RR3_RE1.drop( columns=[RR_Labels[2]], inplace=True)
        H1_RR3_RE2 = H1_RE[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR3_RE2["R_%d" % rR] = H1_RR3_RE2[RR_Labels[2]].cumsum( )
        H1_RR3_RE2.drop( columns=[RR_Labels[2]], inplace=True)
        H1_RR3_RE3 = H1_RE[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR3_RE3["R_%d" % rR] = H1_RR3_RE3[RR_Labels[2]].cumsum( )
        H1_RR3_RE3.drop( columns=[RR_Labels[2]], inplace=True)
        H1_RR3_RE4 = H1_RE[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR3_RE4["R_%d" % rR] = H1_RR3_RE4[RR_Labels[2]].cumsum( )
        H1_RR3_RE4.drop( columns=[RR_Labels[2]], inplace=True)
        # RR 4
        H1_RR4_AET1 = H1_AET[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR4_AET1["R_%d" % rR] = H1_RR4_AET1[RR_Labels[3]].cumsum( )
        H1_RR4_AET1.drop( columns=[RR_Labels[3]], inplace=True)
        H1_RR4_AET2 = H1_AET[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR4_AET2["R_%d" % rR] = H1_RR4_AET2[RR_Labels[3]].cumsum( )
        H1_RR4_AET2.drop( columns=[RR_Labels[3]], inplace=True)
        H1_RR4_AET3 = H1_AET[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR4_AET3["R_%d" % rR] = H1_RR4_AET3[RR_Labels[3]].cumsum( )
        H1_RR4_AET3.drop( columns=[RR_Labels[3]], inplace=True)
        H1_RR4_AET4 = H1_AET[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR4_AET4["R_%d" % rR] = H1_RR4_AET4[RR_Labels[3]].cumsum( )
        H1_RR4_AET4.drop( columns=[RR_Labels[3]], inplace=True)
        H1_RR4_RO1 = H1_RO[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR4_RO1["R_%d" % rR] = H1_RR4_RO1[RR_Labels[3]].cumsum( )
        H1_RR4_RO1.drop( columns=[RR_Labels[3]], inplace=True)
        H1_RR4_RO2 = H1_RO[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR4_RO2["R_%d" % rR] = H1_RR4_RO2[RR_Labels[3]].cumsum( )
        H1_RR4_RO2.drop( columns=[RR_Labels[3]], inplace=True)
        H1_RR4_RO3 = H1_RO[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR4_RO3["R_%d" % rR] = H1_RR4_RO3[RR_Labels[3]].cumsum( )
        H1_RR4_RO3.drop( columns=[RR_Labels[3]], inplace=True)
        H1_RR4_RO4 = H1_RO[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR4_RO4["R_%d" % rR] = H1_RR4_RO4[RR_Labels[3]].cumsum( )
        H1_RR4_RO4.drop( columns=[RR_Labels[3]], inplace=True)
        H1_RR4_RE1 = H1_RE[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR4_RE1["R_%d" % rR] = H1_RR4_RE1[RR_Labels[3]].cumsum( )
        H1_RR4_RE1.drop( columns=[RR_Labels[3]], inplace=True)
        H1_RR4_RE2 = H1_RE[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR4_RE2["R_%d" % rR] = H1_RR4_RE2[RR_Labels[3]].cumsum( )
        H1_RR4_RE2.drop( columns=[RR_Labels[3]], inplace=True)
        H1_RR4_RE3 = H1_RE[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR4_RE3["R_%d" % rR] = H1_RR4_RE3[RR_Labels[3]].cumsum( )
        H1_RR4_RE3.drop( columns=[RR_Labels[3]], inplace=True)
        H1_RR4_RE4 = H1_RE[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR4_RE4["R_%d" % rR] = H1_RR4_RE4[RR_Labels[3]].cumsum( )
        H1_RR4_RE4.drop( columns=[RR_Labels[3]], inplace=True)
        # RR 5
        H1_RR5_AET1 = H1_AET[[RR_Labels[4]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR5_AET1["R_%d" % rR] = H1_RR5_AET1[RR_Labels[4]].cumsum( )
        H1_RR5_AET1.drop( columns=[RR_Labels[4]], inplace=True)
        H1_RR5_AET2 = H1_AET[[RR_Labels[4]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR5_AET2["R_%d" % rR] = H1_RR5_AET2[RR_Labels[4]].cumsum( )
        H1_RR5_AET2.drop( columns=[RR_Labels[4]], inplace=True)
        H1_RR5_AET3 = H1_AET[[RR_Labels[4]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR5_AET3["R_%d" % rR] = H1_RR5_AET3[RR_Labels[4]].cumsum( )
        H1_RR5_AET3.drop( columns=[RR_Labels[4]], inplace=True)
        H1_RR5_AET4 = H1_AET[[RR_Labels[4]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR5_AET4["R_%d" % rR] = H1_RR5_AET4[RR_Labels[4]].cumsum( )
        H1_RR5_AET4.drop( columns=[RR_Labels[4]], inplace=True)
        H1_RR5_RO1 = H1_RO[[RR_Labels[4]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        H1_RR5_RO1["R_%d" % rR] = H1_RR5_RO1[RR_Labels[4]].cumsum( )
        H1_RR5_RO1.drop( columns=[RR_Labels[4]], inplace=True)
        H1_RR5_RO2 = H1_RO[[RR_Labels[4]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        H1_RR5_RO2["R_%d" % rR] = H1_RR5_RO2[RR_Labels[4]].cumsum( )
        H1_RR5_RO2.drop( columns=[RR_Labels[4]], inplace=True)
        H1_RR5_RO3 = H1_RO[[RR_Labels[4]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        H1_RR5_RO3["R_%d" % rR] = H1_RR5_RO3[RR_Labels[4]].cumsum( )
        H1_RR5_RO3.drop( columns=[RR_Labels[4]], inplace=True)
        H1_RR5_RO4 = H1_RO[[RR_Labels[4]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        H1_RR5_RO4["R_%d" % rR] = H1_RR5_RO4[RR_Labels[4]].cumsum( )
        H1_RR5_RO4.drop( columns=[RR_Labels[4]], inplace=True)
    else:
        # HRU 1
        tmpAET = H1_AET[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[0]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[0]], inplace=True )
        H1_HR1_AET1 = H1_HR1_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[0]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[0]], inplace=True )
        H1_HR1_AET2 = H1_HR1_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[0]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[0]], inplace=True )
        H1_HR1_AET3 = H1_HR1_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[0]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[0]], inplace=True )
        H1_HR1_AET4 = H1_HR1_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[0]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[0]], inplace=True )
        H1_HR1_RO1 = H1_HR1_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[0]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[0]], inplace=True )
        H1_HR1_RO2 = H1_HR1_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[0]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[0]], inplace=True )
        H1_HR1_RO3 = H1_HR1_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[0]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[0]], inplace=True )
        H1_HR1_RO4 = H1_HR1_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[HRU_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[0]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[0]], inplace=True )
        H1_HR1_RE1 = H1_HR1_RE1.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[HRU_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[0]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[0]], inplace=True )
        H1_HR1_RE2 = H1_HR1_RE2.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[HRU_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[0]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[0]], inplace=True )
        H1_HR1_RE3 = H1_HR1_RE3.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[HRU_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[0]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[0]], inplace=True )
        H1_HR1_RE4 = H1_HR1_RE4.merge( tmpRE, how='inner', left_index=True, right_index=True )
        # HRU 2
        tmpAET = H1_AET[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[1]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[1]], inplace=True )
        H1_HR2_AET1 = H1_HR2_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[1]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[1]], inplace=True )
        H1_HR2_AET2 = H1_HR2_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[1]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[1]], inplace=True )
        H1_HR2_AET3 = H1_HR2_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[HRU_Labels[1]].cumsum( )
        tmpAET.drop( columns=[HRU_Labels[1]], inplace=True )
        H1_HR2_AET4 = H1_HR2_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[1]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[1]], inplace=True )
        H1_HR2_RO1 = H1_HR2_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[1]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[1]], inplace=True )
        H1_HR2_RO2 = H1_HR2_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[1]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[1]], inplace=True )
        H1_HR2_RO3 = H1_HR2_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[HRU_Labels[1]].cumsum( )
        tmpRO.drop( columns=[HRU_Labels[1]], inplace=True )
        H1_HR2_RO4 = H1_HR2_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[HRU_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[1]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[1]], inplace=True )
        H1_HR2_RE1 = H1_HR2_RE1.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[HRU_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[1]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[1]], inplace=True )
        H1_HR2_RE2 = H1_HR2_RE2.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[HRU_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[1]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[1]], inplace=True )
        H1_HR2_RE3 = H1_HR2_RE3.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[HRU_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[HRU_Labels[1]].cumsum( )
        tmpRE.drop( columns=[HRU_Labels[1]], inplace=True )
        H1_HR2_RE4 = H1_HR2_RE4.merge( tmpRE, how='inner', left_index=True, right_index=True )
        # RR 1
        tmpAET = H1_AET[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[0]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[0]], inplace=True )
        H1_RR1_AET1 = H1_RR1_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[0]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[0]], inplace=True )
        H1_RR1_AET2 = H1_RR1_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[0]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[0]], inplace=True )
        H1_RR1_AET3 = H1_RR1_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[0]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[0]], inplace=True )
        H1_RR1_AET4 = H1_RR1_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[0]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[0]], inplace=True )
        H1_RR1_RO1 = H1_RR1_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[0]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[0]], inplace=True )
        H1_RR1_RO2 = H1_RR1_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[0]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[0]], inplace=True )
        H1_RR1_RO3 = H1_RR1_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[0]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[0]], inplace=True )
        H1_RR1_RO4 = H1_RR1_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[0]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[0]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[0]], inplace=True )
        H1_RR1_RE1 = H1_RR1_RE1.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[0]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[0]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[0]], inplace=True )
        H1_RR1_RE2 = H1_RR1_RE2.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[0]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[0]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[0]], inplace=True )
        H1_RR1_RE3 = H1_RR1_RE3.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[0]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[0]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[0]], inplace=True )
        H1_RR1_RE4 = H1_RR1_RE4.merge( tmpRE, how='inner', left_index=True, right_index=True )
        # RR 2
        tmpAET = H1_AET[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[1]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[1]], inplace=True )
        H1_RR2_AET1 = H1_RR2_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[1]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[1]], inplace=True )
        H1_RR2_AET2 = H1_RR2_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[1]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[1]], inplace=True )
        H1_RR2_AET3 = H1_RR2_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[1]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[1]], inplace=True )
        H1_RR2_AET4 = H1_RR2_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[1]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[1]], inplace=True )
        H1_RR2_RO1 = H1_RR2_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[1]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[1]], inplace=True )
        H1_RR2_RO2 = H1_RR2_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[1]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[1]], inplace=True )
        H1_RR2_RO3 = H1_RR2_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[1]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[1]], inplace=True )
        H1_RR2_RO4 = H1_RR2_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[1]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[1]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[1]], inplace=True )
        H1_RR2_RE1 = H1_RR2_RE1.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[1]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[1]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[1]], inplace=True )
        H1_RR2_RE2 = H1_RR2_RE2.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[1]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[1]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[1]], inplace=True )
        H1_RR2_RE3 = H1_RR2_RE3.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[1]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[1]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[1]], inplace=True )
        H1_RR2_RE4 = H1_RR2_RE4.merge( tmpRE, how='inner', left_index=True, right_index=True )
        # RR 3
        tmpAET = H1_AET[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[2]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[2]], inplace=True )
        H1_RR3_AET1 = H1_RR3_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[2]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[2]], inplace=True )
        H1_RR3_AET2 = H1_RR3_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[2]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[2]], inplace=True )
        H1_RR3_AET3 = H1_RR3_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[2]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[2]], inplace=True )
        H1_RR3_AET4 = H1_RR3_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[2]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[2]], inplace=True )
        H1_RR3_RO1 = H1_RR3_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[2]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[2]], inplace=True )
        H1_RR3_RO2 = H1_RR3_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[2]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[2]], inplace=True )
        H1_RR3_RO3 = H1_RR3_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[2]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[2]], inplace=True )
        H1_RR3_RO4 = H1_RR3_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[2]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[2]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[2]], inplace=True )
        H1_RR3_RE1 = H1_RR3_RE1.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[2]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[2]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[2]], inplace=True )
        H1_RR3_RE2 = H1_RR3_RE2.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[2]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[2]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[2]], inplace=True )
        H1_RR3_RE3 = H1_RR3_RE3.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[2]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[2]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[2]], inplace=True )
        H1_RR3_RE4 = H1_RR3_RE4.merge( tmpRE, how='inner', left_index=True, right_index=True )
        # RR 4
        tmpAET = H1_AET[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[3]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[3]], inplace=True )
        H1_RR4_AET1 = H1_RR4_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[3]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[3]], inplace=True )
        H1_RR4_AET2 = H1_RR4_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[3]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[3]], inplace=True )
        H1_RR4_AET3 = H1_RR4_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[3]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[3]], inplace=True )
        H1_RR4_AET4 = H1_RR4_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[3]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[3]], inplace=True )
        H1_RR4_RO1 = H1_RR4_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[3]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[3]], inplace=True )
        H1_RR4_RO2 = H1_RR4_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[3]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[3]], inplace=True )
        H1_RR4_RO3 = H1_RR4_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[3]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[3]], inplace=True )
        H1_RR4_RO4 = H1_RR4_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[3]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[3]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[3]], inplace=True )
        H1_RR4_RE1 = H1_RR4_RE1.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[3]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[3]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[3]], inplace=True )
        H1_RR4_RE2 = H1_RR4_RE2.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[3]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[3]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[3]], inplace=True )
        H1_RR4_RE3 = H1_RR4_RE3.merge( tmpRE, how='inner', left_index=True, right_index=True )
        tmpRE = H1_RE[[RR_Labels[3]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRE["R_%d" % rR] = tmpRE[RR_Labels[3]].cumsum( )
        tmpRE.drop( columns=[RR_Labels[3]], inplace=True )
        H1_RR4_RE4 = H1_RR4_RE4.merge( tmpRE, how='inner', left_index=True, right_index=True )
        # RR 5
        tmpAET = H1_AET[[RR_Labels[4]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[4]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[4]], inplace=True )
        H1_RR5_AET1 = H1_RR5_AET1.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[4]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[4]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[4]], inplace=True )
        H1_RR5_AET2 = H1_RR5_AET2.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[4]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[4]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[4]], inplace=True )
        H1_RR5_AET3 = H1_RR5_AET3.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpAET = H1_AET[[RR_Labels[4]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpAET["R_%d" % rR] = tmpAET[RR_Labels[4]].cumsum( )
        tmpAET.drop( columns=[RR_Labels[4]], inplace=True )
        H1_RR5_AET4 = H1_RR5_AET4.merge( tmpAET, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[4]]].loc[TP_DICT[1][0]:TP_DICT[1][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[4]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[4]], inplace=True )
        H1_RR5_RO1 = H1_RR5_RO1.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[4]]].loc[TP_DICT[2][0]:TP_DICT[2][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[4]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[4]], inplace=True )
        H1_RR5_RO2 = H1_RR5_RO2.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[4]]].loc[TP_DICT[3][0]:TP_DICT[3][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[4]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[4]], inplace=True )
        H1_RR5_RO3 = H1_RR5_RO3.merge( tmpRO, how='inner', left_index=True, right_index=True )
        tmpRO = H1_RO[[RR_Labels[4]]].loc[TP_DICT[4][0]:TP_DICT[4][1]].copy()
        tmpRO["R_%d" % rR] = tmpRO[RR_Labels[4]].cumsum( )
        tmpRO.drop( columns=[RR_Labels[4]], inplace=True )
        H1_RR5_RO4 = H1_RR5_RO4.merge( tmpRO, how='inner', left_index=True, right_index=True )       
    # end if
# end for

Working on realization 200
Working on realization 400
Working on realization 600
Working on realization 800
Working on realization 1000
Working on realization 1200
Working on realization 1400
Working on realization 1600
Working on realization 1800
Working on realization 2000
Working on realization 2200
Working on realization 2400
Working on realization 2600
Working on realization 2800
Working on realization 3000
Working on realization 3200
Working on realization 3400
Working on realization 3600
Working on realization 3800
Working on realization 4000
Working on realization 4200
Working on realization 4400
Working on realization 4600
Working on realization 4800
Working on realization 5000
Working on realization 5200
Working on realization 5400
Working on realization 5600
Working on realization 5800
Working on realization 6000
Working on realization 6200
Working on realization 6400
Working on realization 6600
Working on realization 6800
Working on realization 7000
Working on realization 7

This loop takes a long time so output this files to pickles now

In [16]:
OFNamer = "Cum_H1_HRU1_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR1_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU1_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR1_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU1_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR1_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU1_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR1_AET4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU2_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR2_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU2_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR2_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU2_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR2_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU2_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR2_AET4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR1_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR1_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR1_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR1_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR1_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR1_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR1_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR1_AET4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR2_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR2_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR2_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR2_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR2_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR2_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR2_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR2_AET4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR3_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR3_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR3_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR3_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR3_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR3_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR3_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR3_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR4_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR4_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR4_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR4_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR4_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR4_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR4_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR4_AET4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR5_AET1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR5_AET1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR5_AET2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR5_AET2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR5_AET3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR5_AET3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR5_AET4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR5_AET4.to_pickle( OFPath, compression='zip' )

In [17]:
OFNamer = "Cum_H1_HRU1_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR1_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU1_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR1_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU1_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR1_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU1_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR1_RO4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU2_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR2_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU2_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR2_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU2_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR2_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU2_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR2_RO4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR1_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR1_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR1_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR1_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR1_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR1_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR1_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR1_RO4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR2_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR2_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR2_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR2_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR2_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR2_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR2_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR2_RO4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR3_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR3_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR3_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR3_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR3_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR3_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR3_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR3_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR4_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR4_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR4_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR4_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR4_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR4_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR4_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR4_RO4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR5_RO1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR5_RO1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR5_RO2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR5_RO2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR5_RO3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR5_RO3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR5_RO4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR5_RO4.to_pickle( OFPath, compression='zip' )

In [18]:
OFNamer = "Cum_H1_HRU1_RE1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR1_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU1_RE2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR1_RE2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU1_RE3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR1_RE3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU1_RE4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR1_RE4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU2_RE1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR2_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU2_RE2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR2_RE2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU2_RE3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR2_RE3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_HRU2_RE4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_HR2_RE4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR1_RE1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR1_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR1_RE2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR1_RE2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR1_RE3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR1_RE3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR1_RE4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR1_RE4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR2_RE1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR2_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR2_RE2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR2_RE2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR2_RE3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR2_RE3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR2_RE4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR2_RE4.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR3_RE1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR3_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR3_RE2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR3_RE2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR3_RE3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR3_RE3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR3_RE4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR3_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR4_RE1_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR4_RE1.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR4_RE2_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR4_RE2.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR4_RE3_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR4_RE3.to_pickle( OFPath, compression='zip' )
OFNamer = "Cum_H1_RR4_RE4_%s.pickle" % OUT_ROOT
OFPath = os.path.normpath( os.path.join( OUT_DIR, OFNamer ) )
H1_RR4_RE4.to_pickle( OFPath, compression='zip' )

## Calculate Time History Stats

Calculate the time history stats now for each DataFrame

### H0

In [19]:
H0_DictHR1AET1 = dict()
H0_DictHR1AET2 = dict()
H0_DictHR1AET3 = dict()
H0_DictHR1AET4 = dict()
H0_DictHR2AET1 = dict()
H0_DictHR2AET2 = dict()
H0_DictHR2AET3 = dict()
H0_DictHR2AET4 = dict()
H0_DictRR1AET1 = dict()
H0_DictRR1AET2 = dict()
H0_DictRR1AET3 = dict()
H0_DictRR1AET4 = dict()
H0_DictRR2AET1 = dict()
H0_DictRR2AET2 = dict()
H0_DictRR2AET3 = dict()
H0_DictRR2AET4 = dict()
H0_DictRR3AET1 = dict()
H0_DictRR3AET2 = dict()
H0_DictRR3AET3 = dict()
H0_DictRR3AET4 = dict()
H0_DictRR4AET1 = dict()
H0_DictRR4AET2 = dict()
H0_DictRR4AET3 = dict()
H0_DictRR4AET4 = dict()
H0_DictRR5AET1 = dict()
H0_DictRR5AET2 = dict()
H0_DictRR5AET3 = dict()
H0_DictRR5AET4 = dict()
H0_DictHR1RO1 = dict()
H0_DictHR1RO2 = dict()
H0_DictHR1RO3 = dict()
H0_DictHR1RO4 = dict()
H0_DictHR2RO1 = dict()
H0_DictHR2RO2 = dict()
H0_DictHR2RO3 = dict()
H0_DictHR2RO4 = dict()
H0_DictRR1RO1 = dict()
H0_DictRR1RO2 = dict()
H0_DictRR1RO3 = dict()
H0_DictRR1RO4 = dict()
H0_DictRR2RO1 = dict()
H0_DictRR2RO2 = dict()
H0_DictRR2RO3 = dict()
H0_DictRR2RO4 = dict()
H0_DictRR3RO1 = dict()
H0_DictRR3RO2 = dict()
H0_DictRR3RO3 = dict()
H0_DictRR3RO4 = dict()
H0_DictRR4RO1 = dict()
H0_DictRR4RO2 = dict()
H0_DictRR4RO3 = dict()
H0_DictRR4RO4 = dict()
H0_DictRR5RO1 = dict()
H0_DictRR5RO2 = dict()
H0_DictRR5RO3 = dict()
H0_DictRR5RO4 = dict()
H0_DictHR1RE1 = dict()
H0_DictHR1RE2 = dict()
H0_DictHR1RE3 = dict()
H0_DictHR1RE4 = dict()
H0_DictHR2RE1 = dict()
H0_DictHR2RE2 = dict()
H0_DictHR2RE3 = dict()
H0_DictHR2RE4 = dict()
H0_DictRR1RE1 = dict()
H0_DictRR1RE2 = dict()
H0_DictRR1RE3 = dict()
H0_DictRR1RE4 = dict()
H0_DictRR2RE1 = dict()
H0_DictRR2RE2 = dict()
H0_DictRR2RE3 = dict()
H0_DictRR2RE4 = dict()
H0_DictRR3RE1 = dict()
H0_DictRR3RE2 = dict()
H0_DictRR3RE3 = dict()
H0_DictRR3RE4 = dict()
H0_DictRR4RE1 = dict()
H0_DictRR4RE2 = dict()
H0_DictRR4RE3 = dict()
H0_DictRR4RE4 = dict()
hdrCnt = 0
# start out by doing the percentiles
for iI in range(NumQs):
    curQ = OurQs[iI]
    H0_DictHR1AET1[StatsHdsList[hdrCnt]] = H0_HR1_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR1AET2[StatsHdsList[hdrCnt]] = H0_HR1_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR1AET3[StatsHdsList[hdrCnt]] = H0_HR1_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR1AET4[StatsHdsList[hdrCnt]] = H0_HR1_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR2AET1[StatsHdsList[hdrCnt]] = H0_HR2_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR2AET2[StatsHdsList[hdrCnt]] = H0_HR2_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR2AET3[StatsHdsList[hdrCnt]] = H0_HR2_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR2AET4[StatsHdsList[hdrCnt]] = H0_HR2_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR1AET1[StatsHdsList[hdrCnt]] = H0_RR1_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR1AET2[StatsHdsList[hdrCnt]] = H0_RR1_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR1AET3[StatsHdsList[hdrCnt]] = H0_RR1_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR1AET4[StatsHdsList[hdrCnt]] = H0_RR1_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR2AET1[StatsHdsList[hdrCnt]] = H0_RR2_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR2AET2[StatsHdsList[hdrCnt]] = H0_RR2_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR2AET3[StatsHdsList[hdrCnt]] = H0_RR2_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR2AET4[StatsHdsList[hdrCnt]] = H0_RR2_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR3AET1[StatsHdsList[hdrCnt]] = H0_RR3_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR3AET2[StatsHdsList[hdrCnt]] = H0_RR3_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR3AET3[StatsHdsList[hdrCnt]] = H0_RR3_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR3AET4[StatsHdsList[hdrCnt]] = H0_RR3_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR4AET1[StatsHdsList[hdrCnt]] = H0_RR4_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR4AET2[StatsHdsList[hdrCnt]] = H0_RR4_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR4AET3[StatsHdsList[hdrCnt]] = H0_RR4_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR4AET4[StatsHdsList[hdrCnt]] = H0_RR4_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR5AET1[StatsHdsList[hdrCnt]] = H0_RR5_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR5AET2[StatsHdsList[hdrCnt]] = H0_RR5_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR5AET3[StatsHdsList[hdrCnt]] = H0_RR5_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR5AET4[StatsHdsList[hdrCnt]] = H0_RR5_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR1RO1[StatsHdsList[hdrCnt]] = H0_HR1_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR1RO2[StatsHdsList[hdrCnt]] = H0_HR1_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR1RO3[StatsHdsList[hdrCnt]] = H0_HR1_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR1RO4[StatsHdsList[hdrCnt]] = H0_HR1_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR2RO1[StatsHdsList[hdrCnt]] = H0_HR2_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR2RO2[StatsHdsList[hdrCnt]] = H0_HR2_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR2RO3[StatsHdsList[hdrCnt]] = H0_HR2_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR2RO4[StatsHdsList[hdrCnt]] = H0_HR2_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR1RO1[StatsHdsList[hdrCnt]] = H0_RR1_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR1RO2[StatsHdsList[hdrCnt]] = H0_RR1_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR1RO3[StatsHdsList[hdrCnt]] = H0_RR1_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR1RO4[StatsHdsList[hdrCnt]] = H0_RR1_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR2RO1[StatsHdsList[hdrCnt]] = H0_RR2_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR2RO2[StatsHdsList[hdrCnt]] = H0_RR2_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR2RO3[StatsHdsList[hdrCnt]] = H0_RR2_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR2RO4[StatsHdsList[hdrCnt]] = H0_RR2_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR3RO1[StatsHdsList[hdrCnt]] = H0_RR3_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR3RO2[StatsHdsList[hdrCnt]] = H0_RR3_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR3RO3[StatsHdsList[hdrCnt]] = H0_RR3_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR3RO4[StatsHdsList[hdrCnt]] = H0_RR3_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR4RO1[StatsHdsList[hdrCnt]] = H0_RR4_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR4RO2[StatsHdsList[hdrCnt]] = H0_RR4_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR4RO3[StatsHdsList[hdrCnt]] = H0_RR4_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR4RO4[StatsHdsList[hdrCnt]] = H0_RR4_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR5RO1[StatsHdsList[hdrCnt]] = H0_RR5_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR5RO2[StatsHdsList[hdrCnt]] = H0_RR5_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR5RO3[StatsHdsList[hdrCnt]] = H0_RR5_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR5RO4[StatsHdsList[hdrCnt]] = H0_RR5_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR1RE1[StatsHdsList[hdrCnt]] = H0_HR1_RE1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR1RE2[StatsHdsList[hdrCnt]] = H0_HR1_RE2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR1RE3[StatsHdsList[hdrCnt]] = H0_HR1_RE3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR1RE4[StatsHdsList[hdrCnt]] = H0_HR1_RE4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR2RE1[StatsHdsList[hdrCnt]] = H0_HR2_RE1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR2RE2[StatsHdsList[hdrCnt]] = H0_HR2_RE2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR2RE3[StatsHdsList[hdrCnt]] = H0_HR2_RE3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictHR2RE4[StatsHdsList[hdrCnt]] = H0_HR2_RE4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR1RE1[StatsHdsList[hdrCnt]] = H0_RR1_RE1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR1RE2[StatsHdsList[hdrCnt]] = H0_RR1_RE2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR1RE3[StatsHdsList[hdrCnt]] = H0_RR1_RE3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR1RE4[StatsHdsList[hdrCnt]] = H0_RR1_RE4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR2RE1[StatsHdsList[hdrCnt]] = H0_RR2_RE1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR2RE2[StatsHdsList[hdrCnt]] = H0_RR2_RE2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR2RE3[StatsHdsList[hdrCnt]] = H0_RR2_RE3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR2RE4[StatsHdsList[hdrCnt]] = H0_RR2_RE4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR3RE1[StatsHdsList[hdrCnt]] = H0_RR3_RE1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR3RE2[StatsHdsList[hdrCnt]] = H0_RR3_RE2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR3RE3[StatsHdsList[hdrCnt]] = H0_RR3_RE3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR3RE4[StatsHdsList[hdrCnt]] = H0_RR3_RE4.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR4RE1[StatsHdsList[hdrCnt]] = H0_RR4_RE1.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR4RE2[StatsHdsList[hdrCnt]] = H0_RR4_RE2.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR4RE3[StatsHdsList[hdrCnt]] = H0_RR4_RE3.quantile( q=curQ, axis=1 ).to_numpy()
    H0_DictRR4RE4[StatsHdsList[hdrCnt]] = H0_RR4_RE4.quantile( q=curQ, axis=1 ).to_numpy()
    hdrCnt += 1
# end for
H0_DictHR1AET1[StatsHdsList[hdrCnt]] = H0_HR1_AET1.mean( axis=1 ).to_numpy()
H0_DictHR1AET2[StatsHdsList[hdrCnt]] = H0_HR1_AET2.mean( axis=1 ).to_numpy()
H0_DictHR1AET3[StatsHdsList[hdrCnt]] = H0_HR1_AET3.mean( axis=1 ).to_numpy()
H0_DictHR1AET4[StatsHdsList[hdrCnt]] = H0_HR1_AET4.mean( axis=1 ).to_numpy()
H0_DictHR2AET1[StatsHdsList[hdrCnt]] = H0_HR2_AET1.mean( axis=1 ).to_numpy()
H0_DictHR2AET2[StatsHdsList[hdrCnt]] = H0_HR2_AET2.mean( axis=1 ).to_numpy()
H0_DictHR2AET3[StatsHdsList[hdrCnt]] = H0_HR2_AET3.mean( axis=1 ).to_numpy()
H0_DictHR2AET4[StatsHdsList[hdrCnt]] = H0_HR2_AET4.mean( axis=1 ).to_numpy()
H0_DictRR1AET1[StatsHdsList[hdrCnt]] = H0_RR1_AET1.mean( axis=1 ).to_numpy()
H0_DictRR1AET2[StatsHdsList[hdrCnt]] = H0_RR1_AET2.mean( axis=1 ).to_numpy()
H0_DictRR1AET3[StatsHdsList[hdrCnt]] = H0_RR1_AET3.mean( axis=1 ).to_numpy()
H0_DictRR1AET4[StatsHdsList[hdrCnt]] = H0_RR1_AET4.mean( axis=1 ).to_numpy()
H0_DictRR2AET1[StatsHdsList[hdrCnt]] = H0_RR2_AET1.mean( axis=1 ).to_numpy()
H0_DictRR2AET2[StatsHdsList[hdrCnt]] = H0_RR2_AET2.mean( axis=1 ).to_numpy()
H0_DictRR2AET3[StatsHdsList[hdrCnt]] = H0_RR2_AET3.mean( axis=1 ).to_numpy()
H0_DictRR2AET4[StatsHdsList[hdrCnt]] = H0_RR2_AET4.mean( axis=1 ).to_numpy()
H0_DictRR3AET1[StatsHdsList[hdrCnt]] = H0_RR3_AET1.mean( axis=1 ).to_numpy()
H0_DictRR3AET2[StatsHdsList[hdrCnt]] = H0_RR3_AET2.mean( axis=1 ).to_numpy()
H0_DictRR3AET3[StatsHdsList[hdrCnt]] = H0_RR3_AET3.mean( axis=1 ).to_numpy()
H0_DictRR3AET4[StatsHdsList[hdrCnt]] = H0_RR3_AET4.mean( axis=1 ).to_numpy()
H0_DictRR4AET1[StatsHdsList[hdrCnt]] = H0_RR4_AET1.mean( axis=1 ).to_numpy()
H0_DictRR4AET2[StatsHdsList[hdrCnt]] = H0_RR4_AET2.mean( axis=1 ).to_numpy()
H0_DictRR4AET3[StatsHdsList[hdrCnt]] = H0_RR4_AET3.mean( axis=1 ).to_numpy()
H0_DictRR4AET4[StatsHdsList[hdrCnt]] = H0_RR4_AET4.mean( axis=1 ).to_numpy()
H0_DictRR5AET1[StatsHdsList[hdrCnt]] = H0_RR5_AET1.mean( axis=1 ).to_numpy()
H0_DictRR5AET2[StatsHdsList[hdrCnt]] = H0_RR5_AET2.mean( axis=1 ).to_numpy()
H0_DictRR5AET3[StatsHdsList[hdrCnt]] = H0_RR5_AET3.mean( axis=1 ).to_numpy()
H0_DictRR5AET4[StatsHdsList[hdrCnt]] = H0_RR5_AET4.mean( axis=1 ).to_numpy()
H0_DictHR1RO1[StatsHdsList[hdrCnt]] = H0_HR1_RO1.mean( axis=1 ).to_numpy()
H0_DictHR1RO2[StatsHdsList[hdrCnt]] = H0_HR1_RO2.mean( axis=1 ).to_numpy()
H0_DictHR1RO3[StatsHdsList[hdrCnt]] = H0_HR1_RO3.mean( axis=1 ).to_numpy()
H0_DictHR1RO4[StatsHdsList[hdrCnt]] = H0_HR1_RO4.mean( axis=1 ).to_numpy()
H0_DictHR2RO1[StatsHdsList[hdrCnt]] = H0_HR2_RO1.mean( axis=1 ).to_numpy()
H0_DictHR2RO2[StatsHdsList[hdrCnt]] = H0_HR2_RO2.mean( axis=1 ).to_numpy()
H0_DictHR2RO3[StatsHdsList[hdrCnt]] = H0_HR2_RO3.mean( axis=1 ).to_numpy()
H0_DictHR2RO4[StatsHdsList[hdrCnt]] = H0_HR2_RO4.mean( axis=1 ).to_numpy()
H0_DictRR1RO1[StatsHdsList[hdrCnt]] = H0_RR1_RO1.mean( axis=1 ).to_numpy()
H0_DictRR1RO2[StatsHdsList[hdrCnt]] = H0_RR1_RO2.mean( axis=1 ).to_numpy()
H0_DictRR1RO3[StatsHdsList[hdrCnt]] = H0_RR1_RO3.mean( axis=1 ).to_numpy()
H0_DictRR1RO4[StatsHdsList[hdrCnt]] = H0_RR1_RO4.mean( axis=1 ).to_numpy()
H0_DictRR2RO1[StatsHdsList[hdrCnt]] = H0_RR2_RO1.mean( axis=1 ).to_numpy()
H0_DictRR2RO2[StatsHdsList[hdrCnt]] = H0_RR2_RO2.mean( axis=1 ).to_numpy()
H0_DictRR2RO3[StatsHdsList[hdrCnt]] = H0_RR2_RO3.mean( axis=1 ).to_numpy()
H0_DictRR2RO4[StatsHdsList[hdrCnt]] = H0_RR2_RO4.mean( axis=1 ).to_numpy()
H0_DictRR3RO1[StatsHdsList[hdrCnt]] = H0_RR3_RO1.mean( axis=1 ).to_numpy()
H0_DictRR3RO2[StatsHdsList[hdrCnt]] = H0_RR3_RO2.mean( axis=1 ).to_numpy()
H0_DictRR3RO3[StatsHdsList[hdrCnt]] = H0_RR3_RO3.mean( axis=1 ).to_numpy()
H0_DictRR3RO4[StatsHdsList[hdrCnt]] = H0_RR3_RO4.mean( axis=1 ).to_numpy()
H0_DictRR4RO1[StatsHdsList[hdrCnt]] = H0_RR4_RO1.mean( axis=1 ).to_numpy()
H0_DictRR4RO2[StatsHdsList[hdrCnt]] = H0_RR4_RO2.mean( axis=1 ).to_numpy()
H0_DictRR4RO3[StatsHdsList[hdrCnt]] = H0_RR4_RO3.mean( axis=1 ).to_numpy()
H0_DictRR4RO4[StatsHdsList[hdrCnt]] = H0_RR4_RO4.mean( axis=1 ).to_numpy()
H0_DictRR5RO1[StatsHdsList[hdrCnt]] = H0_RR5_RO1.mean( axis=1 ).to_numpy()
H0_DictRR5RO2[StatsHdsList[hdrCnt]] = H0_RR5_RO2.mean( axis=1 ).to_numpy()
H0_DictRR5RO3[StatsHdsList[hdrCnt]] = H0_RR5_RO3.mean( axis=1 ).to_numpy()
H0_DictRR5RO4[StatsHdsList[hdrCnt]] = H0_RR5_RO4.mean( axis=1 ).to_numpy()
H0_DictHR1RE1[StatsHdsList[hdrCnt]] = H0_HR1_RE1.mean( axis=1 ).to_numpy()
H0_DictHR1RE2[StatsHdsList[hdrCnt]] = H0_HR1_RE2.mean( axis=1 ).to_numpy()
H0_DictHR1RE3[StatsHdsList[hdrCnt]] = H0_HR1_RE3.mean( axis=1 ).to_numpy()
H0_DictHR1RE4[StatsHdsList[hdrCnt]] = H0_HR1_RE4.mean( axis=1 ).to_numpy()
H0_DictHR2RE1[StatsHdsList[hdrCnt]] = H0_HR2_RE1.mean( axis=1 ).to_numpy()
H0_DictHR2RE2[StatsHdsList[hdrCnt]] = H0_HR2_RE2.mean( axis=1 ).to_numpy()
H0_DictHR2RE3[StatsHdsList[hdrCnt]] = H0_HR2_RE3.mean( axis=1 ).to_numpy()
H0_DictHR2RE4[StatsHdsList[hdrCnt]] = H0_HR2_RE4.mean( axis=1 ).to_numpy()
H0_DictRR1RE1[StatsHdsList[hdrCnt]] = H0_RR1_RE1.mean( axis=1 ).to_numpy()
H0_DictRR1RE2[StatsHdsList[hdrCnt]] = H0_RR1_RE2.mean( axis=1 ).to_numpy()
H0_DictRR1RE3[StatsHdsList[hdrCnt]] = H0_RR1_RE3.mean( axis=1 ).to_numpy()
H0_DictRR1RE4[StatsHdsList[hdrCnt]] = H0_RR1_RE4.mean( axis=1 ).to_numpy()
H0_DictRR2RE1[StatsHdsList[hdrCnt]] = H0_RR2_RE1.mean( axis=1 ).to_numpy()
H0_DictRR2RE2[StatsHdsList[hdrCnt]] = H0_RR2_RE2.mean( axis=1 ).to_numpy()
H0_DictRR2RE3[StatsHdsList[hdrCnt]] = H0_RR2_RE3.mean( axis=1 ).to_numpy()
H0_DictRR2RE4[StatsHdsList[hdrCnt]] = H0_RR2_RE4.mean( axis=1 ).to_numpy()
H0_DictRR3RE1[StatsHdsList[hdrCnt]] = H0_RR3_RE1.mean( axis=1 ).to_numpy()
H0_DictRR3RE2[StatsHdsList[hdrCnt]] = H0_RR3_RE2.mean( axis=1 ).to_numpy()
H0_DictRR3RE3[StatsHdsList[hdrCnt]] = H0_RR3_RE3.mean( axis=1 ).to_numpy()
H0_DictRR3RE4[StatsHdsList[hdrCnt]] = H0_RR3_RE4.mean( axis=1 ).to_numpy()
H0_DictRR4RE1[StatsHdsList[hdrCnt]] = H0_RR4_RE1.mean( axis=1 ).to_numpy()
H0_DictRR4RE2[StatsHdsList[hdrCnt]] = H0_RR4_RE2.mean( axis=1 ).to_numpy()
H0_DictRR4RE3[StatsHdsList[hdrCnt]] = H0_RR4_RE3.mean( axis=1 ).to_numpy()
H0_DictRR4RE4[StatsHdsList[hdrCnt]] = H0_RR4_RE4.mean( axis=1 ).to_numpy()
hdrCnt += 1

In [20]:
H0_DictHR1AET1[StatsHdsList[hdrCnt]] = H0_HR1_AET1.var( axis=1 ).to_numpy()
H0_DictHR1AET2[StatsHdsList[hdrCnt]] = H0_HR1_AET2.var( axis=1 ).to_numpy()
H0_DictHR1AET3[StatsHdsList[hdrCnt]] = H0_HR1_AET3.var( axis=1 ).to_numpy()
H0_DictHR1AET4[StatsHdsList[hdrCnt]] = H0_HR1_AET4.var( axis=1 ).to_numpy()
H0_DictHR2AET1[StatsHdsList[hdrCnt]] = H0_HR2_AET1.var( axis=1 ).to_numpy()
H0_DictHR2AET2[StatsHdsList[hdrCnt]] = H0_HR2_AET2.var( axis=1 ).to_numpy()
H0_DictHR2AET3[StatsHdsList[hdrCnt]] = H0_HR2_AET3.var( axis=1 ).to_numpy()
H0_DictHR2AET4[StatsHdsList[hdrCnt]] = H0_HR2_AET4.var( axis=1 ).to_numpy()
H0_DictRR1AET1[StatsHdsList[hdrCnt]] = H0_RR1_AET1.var( axis=1 ).to_numpy()
H0_DictRR1AET2[StatsHdsList[hdrCnt]] = H0_RR1_AET2.var( axis=1 ).to_numpy()
H0_DictRR1AET3[StatsHdsList[hdrCnt]] = H0_RR1_AET3.var( axis=1 ).to_numpy()
H0_DictRR1AET4[StatsHdsList[hdrCnt]] = H0_RR1_AET4.var( axis=1 ).to_numpy()
H0_DictRR2AET1[StatsHdsList[hdrCnt]] = H0_RR2_AET1.var( axis=1 ).to_numpy()
H0_DictRR2AET2[StatsHdsList[hdrCnt]] = H0_RR2_AET2.var( axis=1 ).to_numpy()
H0_DictRR2AET3[StatsHdsList[hdrCnt]] = H0_RR2_AET3.var( axis=1 ).to_numpy()
H0_DictRR2AET4[StatsHdsList[hdrCnt]] = H0_RR2_AET4.var( axis=1 ).to_numpy()
H0_DictRR3AET1[StatsHdsList[hdrCnt]] = H0_RR3_AET1.var( axis=1 ).to_numpy()
H0_DictRR3AET2[StatsHdsList[hdrCnt]] = H0_RR3_AET2.var( axis=1 ).to_numpy()
H0_DictRR3AET3[StatsHdsList[hdrCnt]] = H0_RR3_AET3.var( axis=1 ).to_numpy()
H0_DictRR3AET4[StatsHdsList[hdrCnt]] = H0_RR3_AET4.var( axis=1 ).to_numpy()
H0_DictRR4AET1[StatsHdsList[hdrCnt]] = H0_RR4_AET1.var( axis=1 ).to_numpy()
H0_DictRR4AET2[StatsHdsList[hdrCnt]] = H0_RR4_AET2.var( axis=1 ).to_numpy()
H0_DictRR4AET3[StatsHdsList[hdrCnt]] = H0_RR4_AET3.var( axis=1 ).to_numpy()
H0_DictRR4AET4[StatsHdsList[hdrCnt]] = H0_RR4_AET4.var( axis=1 ).to_numpy()
H0_DictRR5AET1[StatsHdsList[hdrCnt]] = H0_RR5_AET1.var( axis=1 ).to_numpy()
H0_DictRR5AET2[StatsHdsList[hdrCnt]] = H0_RR5_AET2.var( axis=1 ).to_numpy()
H0_DictRR5AET3[StatsHdsList[hdrCnt]] = H0_RR5_AET3.var( axis=1 ).to_numpy()
H0_DictRR5AET4[StatsHdsList[hdrCnt]] = H0_RR5_AET4.var( axis=1 ).to_numpy()
H0_DictHR1RO1[StatsHdsList[hdrCnt]] = H0_HR1_RO1.var( axis=1 ).to_numpy()
H0_DictHR1RO2[StatsHdsList[hdrCnt]] = H0_HR1_RO2.var( axis=1 ).to_numpy()
H0_DictHR1RO3[StatsHdsList[hdrCnt]] = H0_HR1_RO3.var( axis=1 ).to_numpy()
H0_DictHR1RO4[StatsHdsList[hdrCnt]] = H0_HR1_RO4.var( axis=1 ).to_numpy()
H0_DictHR2RO1[StatsHdsList[hdrCnt]] = H0_HR2_RO1.var( axis=1 ).to_numpy()
H0_DictHR2RO2[StatsHdsList[hdrCnt]] = H0_HR2_RO2.var( axis=1 ).to_numpy()
H0_DictHR2RO3[StatsHdsList[hdrCnt]] = H0_HR2_RO3.var( axis=1 ).to_numpy()
H0_DictHR2RO4[StatsHdsList[hdrCnt]] = H0_HR2_RO4.var( axis=1 ).to_numpy()
H0_DictRR1RO1[StatsHdsList[hdrCnt]] = H0_RR1_RO1.var( axis=1 ).to_numpy()
H0_DictRR1RO2[StatsHdsList[hdrCnt]] = H0_RR1_RO2.var( axis=1 ).to_numpy()
H0_DictRR1RO3[StatsHdsList[hdrCnt]] = H0_RR1_RO3.var( axis=1 ).to_numpy()
H0_DictRR1RO4[StatsHdsList[hdrCnt]] = H0_RR1_RO4.var( axis=1 ).to_numpy()
H0_DictRR2RO1[StatsHdsList[hdrCnt]] = H0_RR2_RO1.var( axis=1 ).to_numpy()
H0_DictRR2RO2[StatsHdsList[hdrCnt]] = H0_RR2_RO2.var( axis=1 ).to_numpy()
H0_DictRR2RO3[StatsHdsList[hdrCnt]] = H0_RR2_RO3.var( axis=1 ).to_numpy()
H0_DictRR2RO4[StatsHdsList[hdrCnt]] = H0_RR2_RO4.var( axis=1 ).to_numpy()
H0_DictRR3RO1[StatsHdsList[hdrCnt]] = H0_RR3_RO1.var( axis=1 ).to_numpy()
H0_DictRR3RO2[StatsHdsList[hdrCnt]] = H0_RR3_RO2.var( axis=1 ).to_numpy()
H0_DictRR3RO3[StatsHdsList[hdrCnt]] = H0_RR3_RO3.var( axis=1 ).to_numpy()
H0_DictRR3RO4[StatsHdsList[hdrCnt]] = H0_RR3_RO4.var( axis=1 ).to_numpy()
H0_DictRR4RO1[StatsHdsList[hdrCnt]] = H0_RR4_RO1.var( axis=1 ).to_numpy()
H0_DictRR4RO2[StatsHdsList[hdrCnt]] = H0_RR4_RO2.var( axis=1 ).to_numpy()
H0_DictRR4RO3[StatsHdsList[hdrCnt]] = H0_RR4_RO3.var( axis=1 ).to_numpy()
H0_DictRR4RO4[StatsHdsList[hdrCnt]] = H0_RR4_RO4.var( axis=1 ).to_numpy()
H0_DictRR5RO1[StatsHdsList[hdrCnt]] = H0_RR5_RO1.var( axis=1 ).to_numpy()
H0_DictRR5RO2[StatsHdsList[hdrCnt]] = H0_RR5_RO2.var( axis=1 ).to_numpy()
H0_DictRR5RO3[StatsHdsList[hdrCnt]] = H0_RR5_RO3.var( axis=1 ).to_numpy()
H0_DictRR5RO4[StatsHdsList[hdrCnt]] = H0_RR5_RO4.var( axis=1 ).to_numpy()
H0_DictHR1RE1[StatsHdsList[hdrCnt]] = H0_HR1_RE1.var( axis=1 ).to_numpy()
H0_DictHR1RE2[StatsHdsList[hdrCnt]] = H0_HR1_RE2.var( axis=1 ).to_numpy()
H0_DictHR1RE3[StatsHdsList[hdrCnt]] = H0_HR1_RE3.var( axis=1 ).to_numpy()
H0_DictHR1RE4[StatsHdsList[hdrCnt]] = H0_HR1_RE4.var( axis=1 ).to_numpy()
H0_DictHR2RE1[StatsHdsList[hdrCnt]] = H0_HR2_RE1.var( axis=1 ).to_numpy()
H0_DictHR2RE2[StatsHdsList[hdrCnt]] = H0_HR2_RE2.var( axis=1 ).to_numpy()
H0_DictHR2RE3[StatsHdsList[hdrCnt]] = H0_HR2_RE3.var( axis=1 ).to_numpy()
H0_DictHR2RE4[StatsHdsList[hdrCnt]] = H0_HR2_RE4.var( axis=1 ).to_numpy()
H0_DictRR1RE1[StatsHdsList[hdrCnt]] = H0_RR1_RE1.var( axis=1 ).to_numpy()
H0_DictRR1RE2[StatsHdsList[hdrCnt]] = H0_RR1_RE2.var( axis=1 ).to_numpy()
H0_DictRR1RE3[StatsHdsList[hdrCnt]] = H0_RR1_RE3.var( axis=1 ).to_numpy()
H0_DictRR1RE4[StatsHdsList[hdrCnt]] = H0_RR1_RE4.var( axis=1 ).to_numpy()
H0_DictRR2RE1[StatsHdsList[hdrCnt]] = H0_RR2_RE1.var( axis=1 ).to_numpy()
H0_DictRR2RE2[StatsHdsList[hdrCnt]] = H0_RR2_RE2.var( axis=1 ).to_numpy()
H0_DictRR2RE3[StatsHdsList[hdrCnt]] = H0_RR2_RE3.var( axis=1 ).to_numpy()
H0_DictRR2RE4[StatsHdsList[hdrCnt]] = H0_RR2_RE4.var( axis=1 ).to_numpy()
H0_DictRR3RE1[StatsHdsList[hdrCnt]] = H0_RR3_RE1.var( axis=1 ).to_numpy()
H0_DictRR3RE2[StatsHdsList[hdrCnt]] = H0_RR3_RE2.var( axis=1 ).to_numpy()
H0_DictRR3RE3[StatsHdsList[hdrCnt]] = H0_RR3_RE3.var( axis=1 ).to_numpy()
H0_DictRR3RE4[StatsHdsList[hdrCnt]] = H0_RR3_RE4.var( axis=1 ).to_numpy()
H0_DictRR4RE1[StatsHdsList[hdrCnt]] = H0_RR4_RE1.var( axis=1 ).to_numpy()
H0_DictRR4RE2[StatsHdsList[hdrCnt]] = H0_RR4_RE2.var( axis=1 ).to_numpy()
H0_DictRR4RE3[StatsHdsList[hdrCnt]] = H0_RR4_RE3.var( axis=1 ).to_numpy()
H0_DictRR4RE4[StatsHdsList[hdrCnt]] = H0_RR4_RE4.var( axis=1 ).to_numpy()
hdrCnt += 1

In [21]:
H0_DictHR1AET1[StatsHdsList[hdrCnt]] = H0_HR1_AET1.skew( axis=1 ).to_numpy()
H0_DictHR1AET2[StatsHdsList[hdrCnt]] = H0_HR1_AET2.skew( axis=1 ).to_numpy()
H0_DictHR1AET3[StatsHdsList[hdrCnt]] = H0_HR1_AET3.skew( axis=1 ).to_numpy()
H0_DictHR1AET4[StatsHdsList[hdrCnt]] = H0_HR1_AET4.skew( axis=1 ).to_numpy()
H0_DictHR2AET1[StatsHdsList[hdrCnt]] = H0_HR2_AET1.skew( axis=1 ).to_numpy()
H0_DictHR2AET2[StatsHdsList[hdrCnt]] = H0_HR2_AET2.skew( axis=1 ).to_numpy()
H0_DictHR2AET3[StatsHdsList[hdrCnt]] = H0_HR2_AET3.skew( axis=1 ).to_numpy()
H0_DictHR2AET4[StatsHdsList[hdrCnt]] = H0_HR2_AET4.skew( axis=1 ).to_numpy()
H0_DictRR1AET1[StatsHdsList[hdrCnt]] = H0_RR1_AET1.skew( axis=1 ).to_numpy()
H0_DictRR1AET2[StatsHdsList[hdrCnt]] = H0_RR1_AET2.skew( axis=1 ).to_numpy()
H0_DictRR1AET3[StatsHdsList[hdrCnt]] = H0_RR1_AET3.skew( axis=1 ).to_numpy()
H0_DictRR1AET4[StatsHdsList[hdrCnt]] = H0_RR1_AET4.skew( axis=1 ).to_numpy()
H0_DictRR2AET1[StatsHdsList[hdrCnt]] = H0_RR2_AET1.skew( axis=1 ).to_numpy()
H0_DictRR2AET2[StatsHdsList[hdrCnt]] = H0_RR2_AET2.skew( axis=1 ).to_numpy()
H0_DictRR2AET3[StatsHdsList[hdrCnt]] = H0_RR2_AET3.skew( axis=1 ).to_numpy()
H0_DictRR2AET4[StatsHdsList[hdrCnt]] = H0_RR2_AET4.skew( axis=1 ).to_numpy()
H0_DictRR3AET1[StatsHdsList[hdrCnt]] = H0_RR3_AET1.skew( axis=1 ).to_numpy()
H0_DictRR3AET2[StatsHdsList[hdrCnt]] = H0_RR3_AET2.skew( axis=1 ).to_numpy()
H0_DictRR3AET3[StatsHdsList[hdrCnt]] = H0_RR3_AET3.skew( axis=1 ).to_numpy()
H0_DictRR3AET4[StatsHdsList[hdrCnt]] = H0_RR3_AET4.skew( axis=1 ).to_numpy()
H0_DictRR4AET1[StatsHdsList[hdrCnt]] = H0_RR4_AET1.skew( axis=1 ).to_numpy()
H0_DictRR4AET2[StatsHdsList[hdrCnt]] = H0_RR4_AET2.skew( axis=1 ).to_numpy()
H0_DictRR4AET3[StatsHdsList[hdrCnt]] = H0_RR4_AET3.skew( axis=1 ).to_numpy()
H0_DictRR4AET4[StatsHdsList[hdrCnt]] = H0_RR4_AET4.skew( axis=1 ).to_numpy()
H0_DictRR5AET1[StatsHdsList[hdrCnt]] = H0_RR5_AET1.skew( axis=1 ).to_numpy()
H0_DictRR5AET2[StatsHdsList[hdrCnt]] = H0_RR5_AET2.skew( axis=1 ).to_numpy()
H0_DictRR5AET3[StatsHdsList[hdrCnt]] = H0_RR5_AET3.skew( axis=1 ).to_numpy()
H0_DictRR5AET4[StatsHdsList[hdrCnt]] = H0_RR5_AET4.skew( axis=1 ).to_numpy()
H0_DictHR1RO1[StatsHdsList[hdrCnt]] = H0_HR1_RO1.skew( axis=1 ).to_numpy()
H0_DictHR1RO2[StatsHdsList[hdrCnt]] = H0_HR1_RO2.skew( axis=1 ).to_numpy()
H0_DictHR1RO3[StatsHdsList[hdrCnt]] = H0_HR1_RO3.skew( axis=1 ).to_numpy()
H0_DictHR1RO4[StatsHdsList[hdrCnt]] = H0_HR1_RO4.skew( axis=1 ).to_numpy()
H0_DictHR2RO1[StatsHdsList[hdrCnt]] = H0_HR2_RO1.skew( axis=1 ).to_numpy()
H0_DictHR2RO2[StatsHdsList[hdrCnt]] = H0_HR2_RO2.skew( axis=1 ).to_numpy()
H0_DictHR2RO3[StatsHdsList[hdrCnt]] = H0_HR2_RO3.skew( axis=1 ).to_numpy()
H0_DictHR2RO4[StatsHdsList[hdrCnt]] = H0_HR2_RO4.skew( axis=1 ).to_numpy()
H0_DictRR1RO1[StatsHdsList[hdrCnt]] = H0_RR1_RO1.skew( axis=1 ).to_numpy()
H0_DictRR1RO2[StatsHdsList[hdrCnt]] = H0_RR1_RO2.skew( axis=1 ).to_numpy()
H0_DictRR1RO3[StatsHdsList[hdrCnt]] = H0_RR1_RO3.skew( axis=1 ).to_numpy()
H0_DictRR1RO4[StatsHdsList[hdrCnt]] = H0_RR1_RO4.skew( axis=1 ).to_numpy()
H0_DictRR2RO1[StatsHdsList[hdrCnt]] = H0_RR2_RO1.skew( axis=1 ).to_numpy()
H0_DictRR2RO2[StatsHdsList[hdrCnt]] = H0_RR2_RO2.skew( axis=1 ).to_numpy()
H0_DictRR2RO3[StatsHdsList[hdrCnt]] = H0_RR2_RO3.skew( axis=1 ).to_numpy()
H0_DictRR2RO4[StatsHdsList[hdrCnt]] = H0_RR2_RO4.skew( axis=1 ).to_numpy()
H0_DictRR3RO1[StatsHdsList[hdrCnt]] = H0_RR3_RO1.skew( axis=1 ).to_numpy()
H0_DictRR3RO2[StatsHdsList[hdrCnt]] = H0_RR3_RO2.skew( axis=1 ).to_numpy()
H0_DictRR3RO3[StatsHdsList[hdrCnt]] = H0_RR3_RO3.skew( axis=1 ).to_numpy()
H0_DictRR3RO4[StatsHdsList[hdrCnt]] = H0_RR3_RO4.skew( axis=1 ).to_numpy()
H0_DictRR4RO1[StatsHdsList[hdrCnt]] = H0_RR4_RO1.skew( axis=1 ).to_numpy()
H0_DictRR4RO2[StatsHdsList[hdrCnt]] = H0_RR4_RO2.skew( axis=1 ).to_numpy()
H0_DictRR4RO3[StatsHdsList[hdrCnt]] = H0_RR4_RO3.skew( axis=1 ).to_numpy()
H0_DictRR4RO4[StatsHdsList[hdrCnt]] = H0_RR4_RO4.skew( axis=1 ).to_numpy()
H0_DictRR5RO1[StatsHdsList[hdrCnt]] = H0_RR5_RO1.skew( axis=1 ).to_numpy()
H0_DictRR5RO2[StatsHdsList[hdrCnt]] = H0_RR5_RO2.skew( axis=1 ).to_numpy()
H0_DictRR5RO3[StatsHdsList[hdrCnt]] = H0_RR5_RO3.skew( axis=1 ).to_numpy()
H0_DictRR5RO4[StatsHdsList[hdrCnt]] = H0_RR5_RO4.skew( axis=1 ).to_numpy()
H0_DictHR1RE1[StatsHdsList[hdrCnt]] = H0_HR1_RE1.skew( axis=1 ).to_numpy()
H0_DictHR1RE2[StatsHdsList[hdrCnt]] = H0_HR1_RE2.skew( axis=1 ).to_numpy()
H0_DictHR1RE3[StatsHdsList[hdrCnt]] = H0_HR1_RE3.skew( axis=1 ).to_numpy()
H0_DictHR1RE4[StatsHdsList[hdrCnt]] = H0_HR1_RE4.skew( axis=1 ).to_numpy()
H0_DictHR2RE1[StatsHdsList[hdrCnt]] = H0_HR2_RE1.skew( axis=1 ).to_numpy()
H0_DictHR2RE2[StatsHdsList[hdrCnt]] = H0_HR2_RE2.skew( axis=1 ).to_numpy()
H0_DictHR2RE3[StatsHdsList[hdrCnt]] = H0_HR2_RE3.skew( axis=1 ).to_numpy()
H0_DictHR2RE4[StatsHdsList[hdrCnt]] = H0_HR2_RE4.skew( axis=1 ).to_numpy()
H0_DictRR1RE1[StatsHdsList[hdrCnt]] = H0_RR1_RE1.skew( axis=1 ).to_numpy()
H0_DictRR1RE2[StatsHdsList[hdrCnt]] = H0_RR1_RE2.skew( axis=1 ).to_numpy()
H0_DictRR1RE3[StatsHdsList[hdrCnt]] = H0_RR1_RE3.skew( axis=1 ).to_numpy()
H0_DictRR1RE4[StatsHdsList[hdrCnt]] = H0_RR1_RE4.skew( axis=1 ).to_numpy()
H0_DictRR2RE1[StatsHdsList[hdrCnt]] = H0_RR2_RE1.skew( axis=1 ).to_numpy()
H0_DictRR2RE2[StatsHdsList[hdrCnt]] = H0_RR2_RE2.skew( axis=1 ).to_numpy()
H0_DictRR2RE3[StatsHdsList[hdrCnt]] = H0_RR2_RE3.skew( axis=1 ).to_numpy()
H0_DictRR2RE4[StatsHdsList[hdrCnt]] = H0_RR2_RE4.skew( axis=1 ).to_numpy()
H0_DictRR3RE1[StatsHdsList[hdrCnt]] = H0_RR3_RE1.skew( axis=1 ).to_numpy()
H0_DictRR3RE2[StatsHdsList[hdrCnt]] = H0_RR3_RE2.skew( axis=1 ).to_numpy()
H0_DictRR3RE3[StatsHdsList[hdrCnt]] = H0_RR3_RE3.skew( axis=1 ).to_numpy()
H0_DictRR3RE4[StatsHdsList[hdrCnt]] = H0_RR3_RE4.skew( axis=1 ).to_numpy()
H0_DictRR4RE1[StatsHdsList[hdrCnt]] = H0_RR4_RE1.skew( axis=1 ).to_numpy()
H0_DictRR4RE2[StatsHdsList[hdrCnt]] = H0_RR4_RE2.skew( axis=1 ).to_numpy()
H0_DictRR4RE3[StatsHdsList[hdrCnt]] = H0_RR4_RE3.skew( axis=1 ).to_numpy()
H0_DictRR4RE4[StatsHdsList[hdrCnt]] = H0_RR4_RE4.skew( axis=1 ).to_numpy()
hdrCnt += 1

In [22]:
H0_DictHR1AET1[StatsHdsList[hdrCnt]] = H0_HR1_AET1.kurt( axis=1 ).to_numpy()
H0_DictHR1AET2[StatsHdsList[hdrCnt]] = H0_HR1_AET2.kurt( axis=1 ).to_numpy()
H0_DictHR1AET3[StatsHdsList[hdrCnt]] = H0_HR1_AET3.kurt( axis=1 ).to_numpy()
H0_DictHR1AET4[StatsHdsList[hdrCnt]] = H0_HR1_AET4.kurt( axis=1 ).to_numpy()
H0_DictHR2AET1[StatsHdsList[hdrCnt]] = H0_HR2_AET1.kurt( axis=1 ).to_numpy()
H0_DictHR2AET2[StatsHdsList[hdrCnt]] = H0_HR2_AET2.kurt( axis=1 ).to_numpy()
H0_DictHR2AET3[StatsHdsList[hdrCnt]] = H0_HR2_AET3.kurt( axis=1 ).to_numpy()
H0_DictHR2AET4[StatsHdsList[hdrCnt]] = H0_HR2_AET4.kurt( axis=1 ).to_numpy()
H0_DictRR1AET1[StatsHdsList[hdrCnt]] = H0_RR1_AET1.kurt( axis=1 ).to_numpy()
H0_DictRR1AET2[StatsHdsList[hdrCnt]] = H0_RR1_AET2.kurt( axis=1 ).to_numpy()
H0_DictRR1AET3[StatsHdsList[hdrCnt]] = H0_RR1_AET3.kurt( axis=1 ).to_numpy()
H0_DictRR1AET4[StatsHdsList[hdrCnt]] = H0_RR1_AET4.kurt( axis=1 ).to_numpy()
H0_DictRR2AET1[StatsHdsList[hdrCnt]] = H0_RR2_AET1.kurt( axis=1 ).to_numpy()
H0_DictRR2AET2[StatsHdsList[hdrCnt]] = H0_RR2_AET2.kurt( axis=1 ).to_numpy()
H0_DictRR2AET3[StatsHdsList[hdrCnt]] = H0_RR2_AET3.kurt( axis=1 ).to_numpy()
H0_DictRR2AET4[StatsHdsList[hdrCnt]] = H0_RR2_AET4.kurt( axis=1 ).to_numpy()
H0_DictRR3AET1[StatsHdsList[hdrCnt]] = H0_RR3_AET1.kurt( axis=1 ).to_numpy()
H0_DictRR3AET2[StatsHdsList[hdrCnt]] = H0_RR3_AET2.kurt( axis=1 ).to_numpy()
H0_DictRR3AET3[StatsHdsList[hdrCnt]] = H0_RR3_AET3.kurt( axis=1 ).to_numpy()
H0_DictRR3AET4[StatsHdsList[hdrCnt]] = H0_RR3_AET4.kurt( axis=1 ).to_numpy()
H0_DictRR4AET1[StatsHdsList[hdrCnt]] = H0_RR4_AET1.kurt( axis=1 ).to_numpy()
H0_DictRR4AET2[StatsHdsList[hdrCnt]] = H0_RR4_AET2.kurt( axis=1 ).to_numpy()
H0_DictRR4AET3[StatsHdsList[hdrCnt]] = H0_RR4_AET3.kurt( axis=1 ).to_numpy()
H0_DictRR4AET4[StatsHdsList[hdrCnt]] = H0_RR4_AET4.kurt( axis=1 ).to_numpy()
H0_DictRR5AET1[StatsHdsList[hdrCnt]] = H0_RR5_AET1.kurt( axis=1 ).to_numpy()
H0_DictRR5AET2[StatsHdsList[hdrCnt]] = H0_RR5_AET2.kurt( axis=1 ).to_numpy()
H0_DictRR5AET3[StatsHdsList[hdrCnt]] = H0_RR5_AET3.kurt( axis=1 ).to_numpy()
H0_DictRR5AET4[StatsHdsList[hdrCnt]] = H0_RR5_AET4.kurt( axis=1 ).to_numpy()
H0_DictHR1RO1[StatsHdsList[hdrCnt]] = H0_HR1_RO1.kurt( axis=1 ).to_numpy()
H0_DictHR1RO2[StatsHdsList[hdrCnt]] = H0_HR1_RO2.kurt( axis=1 ).to_numpy()
H0_DictHR1RO3[StatsHdsList[hdrCnt]] = H0_HR1_RO3.kurt( axis=1 ).to_numpy()
H0_DictHR1RO4[StatsHdsList[hdrCnt]] = H0_HR1_RO4.kurt( axis=1 ).to_numpy()
H0_DictHR2RO1[StatsHdsList[hdrCnt]] = H0_HR2_RO1.kurt( axis=1 ).to_numpy()
H0_DictHR2RO2[StatsHdsList[hdrCnt]] = H0_HR2_RO2.kurt( axis=1 ).to_numpy()
H0_DictHR2RO3[StatsHdsList[hdrCnt]] = H0_HR2_RO3.kurt( axis=1 ).to_numpy()
H0_DictHR2RO4[StatsHdsList[hdrCnt]] = H0_HR2_RO4.kurt( axis=1 ).to_numpy()
H0_DictRR1RO1[StatsHdsList[hdrCnt]] = H0_RR1_RO1.kurt( axis=1 ).to_numpy()
H0_DictRR1RO2[StatsHdsList[hdrCnt]] = H0_RR1_RO2.kurt( axis=1 ).to_numpy()
H0_DictRR1RO3[StatsHdsList[hdrCnt]] = H0_RR1_RO3.kurt( axis=1 ).to_numpy()
H0_DictRR1RO4[StatsHdsList[hdrCnt]] = H0_RR1_RO4.kurt( axis=1 ).to_numpy()
H0_DictRR2RO1[StatsHdsList[hdrCnt]] = H0_RR2_RO1.kurt( axis=1 ).to_numpy()
H0_DictRR2RO2[StatsHdsList[hdrCnt]] = H0_RR2_RO2.kurt( axis=1 ).to_numpy()
H0_DictRR2RO3[StatsHdsList[hdrCnt]] = H0_RR2_RO3.kurt( axis=1 ).to_numpy()
H0_DictRR2RO4[StatsHdsList[hdrCnt]] = H0_RR2_RO4.kurt( axis=1 ).to_numpy()
H0_DictRR3RO1[StatsHdsList[hdrCnt]] = H0_RR3_RO1.kurt( axis=1 ).to_numpy()
H0_DictRR3RO2[StatsHdsList[hdrCnt]] = H0_RR3_RO2.kurt( axis=1 ).to_numpy()
H0_DictRR3RO3[StatsHdsList[hdrCnt]] = H0_RR3_RO3.kurt( axis=1 ).to_numpy()
H0_DictRR3RO4[StatsHdsList[hdrCnt]] = H0_RR3_RO4.kurt( axis=1 ).to_numpy()
H0_DictRR4RO1[StatsHdsList[hdrCnt]] = H0_RR4_RO1.kurt( axis=1 ).to_numpy()
H0_DictRR4RO2[StatsHdsList[hdrCnt]] = H0_RR4_RO2.kurt( axis=1 ).to_numpy()
H0_DictRR4RO3[StatsHdsList[hdrCnt]] = H0_RR4_RO3.kurt( axis=1 ).to_numpy()
H0_DictRR4RO4[StatsHdsList[hdrCnt]] = H0_RR4_RO4.kurt( axis=1 ).to_numpy()
H0_DictRR5RO1[StatsHdsList[hdrCnt]] = H0_RR5_RO1.kurt( axis=1 ).to_numpy()
H0_DictRR5RO2[StatsHdsList[hdrCnt]] = H0_RR5_RO2.kurt( axis=1 ).to_numpy()
H0_DictRR5RO3[StatsHdsList[hdrCnt]] = H0_RR5_RO3.kurt( axis=1 ).to_numpy()
H0_DictRR5RO4[StatsHdsList[hdrCnt]] = H0_RR5_RO4.kurt( axis=1 ).to_numpy()
H0_DictHR1RE1[StatsHdsList[hdrCnt]] = H0_HR1_RE1.kurt( axis=1 ).to_numpy()
H0_DictHR1RE2[StatsHdsList[hdrCnt]] = H0_HR1_RE2.kurt( axis=1 ).to_numpy()
H0_DictHR1RE3[StatsHdsList[hdrCnt]] = H0_HR1_RE3.kurt( axis=1 ).to_numpy()
H0_DictHR1RE4[StatsHdsList[hdrCnt]] = H0_HR1_RE4.kurt( axis=1 ).to_numpy()
H0_DictHR2RE1[StatsHdsList[hdrCnt]] = H0_HR2_RE1.kurt( axis=1 ).to_numpy()
H0_DictHR2RE2[StatsHdsList[hdrCnt]] = H0_HR2_RE2.kurt( axis=1 ).to_numpy()
H0_DictHR2RE3[StatsHdsList[hdrCnt]] = H0_HR2_RE3.kurt( axis=1 ).to_numpy()
H0_DictHR2RE4[StatsHdsList[hdrCnt]] = H0_HR2_RE4.kurt( axis=1 ).to_numpy()
H0_DictRR1RE1[StatsHdsList[hdrCnt]] = H0_RR1_RE1.kurt( axis=1 ).to_numpy()
H0_DictRR1RE2[StatsHdsList[hdrCnt]] = H0_RR1_RE2.kurt( axis=1 ).to_numpy()
H0_DictRR1RE3[StatsHdsList[hdrCnt]] = H0_RR1_RE3.kurt( axis=1 ).to_numpy()
H0_DictRR1RE4[StatsHdsList[hdrCnt]] = H0_RR1_RE4.kurt( axis=1 ).to_numpy()
H0_DictRR2RE1[StatsHdsList[hdrCnt]] = H0_RR2_RE1.kurt( axis=1 ).to_numpy()
H0_DictRR2RE2[StatsHdsList[hdrCnt]] = H0_RR2_RE2.kurt( axis=1 ).to_numpy()
H0_DictRR2RE3[StatsHdsList[hdrCnt]] = H0_RR2_RE3.kurt( axis=1 ).to_numpy()
H0_DictRR2RE4[StatsHdsList[hdrCnt]] = H0_RR2_RE4.kurt( axis=1 ).to_numpy()
H0_DictRR3RE1[StatsHdsList[hdrCnt]] = H0_RR3_RE1.kurt( axis=1 ).to_numpy()
H0_DictRR3RE2[StatsHdsList[hdrCnt]] = H0_RR3_RE2.kurt( axis=1 ).to_numpy()
H0_DictRR3RE3[StatsHdsList[hdrCnt]] = H0_RR3_RE3.kurt( axis=1 ).to_numpy()
H0_DictRR3RE4[StatsHdsList[hdrCnt]] = H0_RR3_RE4.kurt( axis=1 ).to_numpy()
H0_DictRR4RE1[StatsHdsList[hdrCnt]] = H0_RR4_RE1.kurt( axis=1 ).to_numpy()
H0_DictRR4RE2[StatsHdsList[hdrCnt]] = H0_RR4_RE2.kurt( axis=1 ).to_numpy()
H0_DictRR4RE3[StatsHdsList[hdrCnt]] = H0_RR4_RE3.kurt( axis=1 ).to_numpy()
H0_DictRR4RE4[StatsHdsList[hdrCnt]] = H0_RR4_RE4.kurt( axis=1 ).to_numpy()

Make our DataFrames from the Stats

In [23]:
TimIndex1 = H0_HR1_AET1.index
TimIndex2 = H0_HR1_AET2.index
TimIndex3 = H0_HR1_AET3.index
TimIndex4 = H0_HR1_AET4.index

In [24]:
H0_HR1AETStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictHR1AET1 )
H0_HR1AETStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictHR1AET2 )
H0_HR1AETStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictHR1AET3 )
H0_HR1AETStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictHR1AET4 )
H0_HR2AETStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictHR2AET1 )
H0_HR2AETStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictHR2AET2 )
H0_HR2AETStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictHR2AET3 )
H0_HR2AETStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictHR2AET4 )
H0_RR1AETStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR1AET1 )
H0_RR1AETStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR1AET2 )
H0_RR1AETStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR1AET3 )
H0_RR1AETStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR1AET4 )
H0_RR2AETStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR2AET1 )
H0_RR2AETStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR2AET2 )
H0_RR2AETStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR2AET3 )
H0_RR2AETStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR2AET4 )
H0_RR3AETStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR3AET1 )
H0_RR3AETStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR3AET2 )
H0_RR3AETStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR3AET3 )
H0_RR3AETStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR3AET4 )
H0_RR4AETStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR4AET1 )
H0_RR4AETStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR4AET2 )
H0_RR4AETStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR4AET3 )
H0_RR4AETStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR4AET4 )
H0_RR5AETStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR5AET1 )
H0_RR5AETStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR5AET2 )
H0_RR5AETStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR5AET3 )
H0_RR5AETStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR5AET4 )

In [25]:
H0_HR1ROStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictHR1RO1 )
H0_HR1ROStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictHR1RO2 )
H0_HR1ROStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictHR1RO3 )
H0_HR1ROStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictHR1RO4 )
H0_HR2ROStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictHR2RO1 )
H0_HR2ROStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictHR2RO2 )
H0_HR2ROStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictHR2RO3 )
H0_HR2ROStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictHR2RO4 )
H0_RR1ROStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR1RO1 )
H0_RR1ROStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR1RO2 )
H0_RR1ROStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR1RO3 )
H0_RR1ROStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR1RO4 )
H0_RR2ROStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR2RO1 )
H0_RR2ROStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR2RO2 )
H0_RR2ROStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR2RO3 )
H0_RR2ROStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR2RO4 )
H0_RR3ROStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR3RO1 )
H0_RR3ROStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR3RO2 )
H0_RR3ROStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR3RO3 )
H0_RR3ROStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR3RO4 )
H0_RR4ROStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR4RO1 )
H0_RR4ROStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR4RO2 )
H0_RR4ROStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR4RO3 )
H0_RR4ROStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR4RO4 )
H0_RR5ROStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR5RO1 )
H0_RR5ROStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR5RO2 )
H0_RR5ROStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR5RO3 )
H0_RR5ROStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR5RO4 )

In [26]:
H0_HR1REStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictHR1RE1 )
H0_HR1REStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictHR1RE2 )
H0_HR1REStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictHR1RE3 )
H0_HR1REStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictHR1RE4 )
H0_HR2REStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictHR2RE1 )
H0_HR2REStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictHR2RE2 )
H0_HR2REStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictHR2RE3 )
H0_HR2REStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictHR2RE4 )
H0_RR1REStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR1RE1 )
H0_RR1REStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR1RE2 )
H0_RR1REStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR1RE3 )
H0_RR1REStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR1RE4 )
H0_RR2REStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR2RE1 )
H0_RR2REStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR2RE2 )
H0_RR2REStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR2RE3 )
H0_RR2REStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR2RE4 )
H0_RR3REStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR3RE1 )
H0_RR3REStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR3RE2 )
H0_RR3REStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR3RE3 )
H0_RR3REStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR3RE4 )
H0_RR4REStats1 = pd.DataFrame(index=TimIndex1, data=H0_DictRR4RE1 )
H0_RR4REStats2 = pd.DataFrame(index=TimIndex2, data=H0_DictRR4RE2 )
H0_RR4REStats3 = pd.DataFrame(index=TimIndex3, data=H0_DictRR4RE3 )
H0_RR4REStats4 = pd.DataFrame(index=TimIndex4, data=H0_DictRR4RE4 )

Output to a spreadsheets

In [27]:
OutFileName = "Cum_H0_HR1_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H0_HR1AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR1AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR1AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR1AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR1ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR1ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR1ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR1ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR1REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR1REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR1REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR1REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [28]:
OutFileName = "Cum_H0_HR2_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H0_HR2AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR2AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR2AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR2AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR2ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR2ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR2ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR2ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR2REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR2REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR2REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_HR2REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [29]:
OutFileName = "Cum_H0_RR1_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H0_RR1AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR1AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR1AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR1AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR1ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR1ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR1ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR1ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR1REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR1REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR1REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR1REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [30]:
OutFileName = "Cum_H0_RR2_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H0_RR2AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR2AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR2AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR2AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR2ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR2ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR2ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR2ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR2REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR2REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR2REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR2REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [31]:
OutFileName = "Cum_H0_RR3_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H0_RR3AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR3AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR3AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR3AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR3ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR3ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR3ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR3ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR3REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR3REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR3REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR3REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [32]:
OutFileName = "Cum_H0_RR4_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H0_RR4AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR4AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR4AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR4AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR4ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR4ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR4ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR4ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR4REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR4REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR4REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR4REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [33]:
OutFileName = "Cum_H0_RR5_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H0_RR5AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR5AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR5AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR5AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR5ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR5ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR5ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H0_RR5ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with

### H1

In [34]:
H1_DictHR1AET1 = dict()
H1_DictHR1AET2 = dict()
H1_DictHR1AET3 = dict()
H1_DictHR1AET4 = dict()
H1_DictHR2AET1 = dict()
H1_DictHR2AET2 = dict()
H1_DictHR2AET3 = dict()
H1_DictHR2AET4 = dict()
H1_DictRR1AET1 = dict()
H1_DictRR1AET2 = dict()
H1_DictRR1AET3 = dict()
H1_DictRR1AET4 = dict()
H1_DictRR2AET1 = dict()
H1_DictRR2AET2 = dict()
H1_DictRR2AET3 = dict()
H1_DictRR2AET4 = dict()
H1_DictRR3AET1 = dict()
H1_DictRR3AET2 = dict()
H1_DictRR3AET3 = dict()
H1_DictRR3AET4 = dict()
H1_DictRR4AET1 = dict()
H1_DictRR4AET2 = dict()
H1_DictRR4AET3 = dict()
H1_DictRR4AET4 = dict()
H1_DictRR5AET1 = dict()
H1_DictRR5AET2 = dict()
H1_DictRR5AET3 = dict()
H1_DictRR5AET4 = dict()
H1_DictHR1RO1 = dict()
H1_DictHR1RO2 = dict()
H1_DictHR1RO3 = dict()
H1_DictHR1RO4 = dict()
H1_DictHR2RO1 = dict()
H1_DictHR2RO2 = dict()
H1_DictHR2RO3 = dict()
H1_DictHR2RO4 = dict()
H1_DictRR1RO1 = dict()
H1_DictRR1RO2 = dict()
H1_DictRR1RO3 = dict()
H1_DictRR1RO4 = dict()
H1_DictRR2RO1 = dict()
H1_DictRR2RO2 = dict()
H1_DictRR2RO3 = dict()
H1_DictRR2RO4 = dict()
H1_DictRR3RO1 = dict()
H1_DictRR3RO2 = dict()
H1_DictRR3RO3 = dict()
H1_DictRR3RO4 = dict()
H1_DictRR4RO1 = dict()
H1_DictRR4RO2 = dict()
H1_DictRR4RO3 = dict()
H1_DictRR4RO4 = dict()
H1_DictRR5RO1 = dict()
H1_DictRR5RO2 = dict()
H1_DictRR5RO3 = dict()
H1_DictRR5RO4 = dict()
H1_DictHR1RE1 = dict()
H1_DictHR1RE2 = dict()
H1_DictHR1RE3 = dict()
H1_DictHR1RE4 = dict()
H1_DictHR2RE1 = dict()
H1_DictHR2RE2 = dict()
H1_DictHR2RE3 = dict()
H1_DictHR2RE4 = dict()
H1_DictRR1RE1 = dict()
H1_DictRR1RE2 = dict()
H1_DictRR1RE3 = dict()
H1_DictRR1RE4 = dict()
H1_DictRR2RE1 = dict()
H1_DictRR2RE2 = dict()
H1_DictRR2RE3 = dict()
H1_DictRR2RE4 = dict()
H1_DictRR3RE1 = dict()
H1_DictRR3RE2 = dict()
H1_DictRR3RE3 = dict()
H1_DictRR3RE4 = dict()
H1_DictRR4RE1 = dict()
H1_DictRR4RE2 = dict()
H1_DictRR4RE3 = dict()
H1_DictRR4RE4 = dict()
hdrCnt = 0
# start out by doing the percentiles
for iI in range(NumQs):
    curQ = OurQs[iI]
    H1_DictHR1AET1[StatsHdsList[hdrCnt]] = H1_HR1_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR1AET2[StatsHdsList[hdrCnt]] = H1_HR1_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR1AET3[StatsHdsList[hdrCnt]] = H1_HR1_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR1AET4[StatsHdsList[hdrCnt]] = H1_HR1_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR2AET1[StatsHdsList[hdrCnt]] = H1_HR2_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR2AET2[StatsHdsList[hdrCnt]] = H1_HR2_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR2AET3[StatsHdsList[hdrCnt]] = H1_HR2_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR2AET4[StatsHdsList[hdrCnt]] = H1_HR2_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR1AET1[StatsHdsList[hdrCnt]] = H1_RR1_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR1AET2[StatsHdsList[hdrCnt]] = H1_RR1_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR1AET3[StatsHdsList[hdrCnt]] = H1_RR1_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR1AET4[StatsHdsList[hdrCnt]] = H1_RR1_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR2AET1[StatsHdsList[hdrCnt]] = H1_RR2_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR2AET2[StatsHdsList[hdrCnt]] = H1_RR2_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR2AET3[StatsHdsList[hdrCnt]] = H1_RR2_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR2AET4[StatsHdsList[hdrCnt]] = H1_RR2_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR3AET1[StatsHdsList[hdrCnt]] = H1_RR3_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR3AET2[StatsHdsList[hdrCnt]] = H1_RR3_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR3AET3[StatsHdsList[hdrCnt]] = H1_RR3_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR3AET4[StatsHdsList[hdrCnt]] = H1_RR3_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR4AET1[StatsHdsList[hdrCnt]] = H1_RR4_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR4AET2[StatsHdsList[hdrCnt]] = H1_RR4_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR4AET3[StatsHdsList[hdrCnt]] = H1_RR4_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR4AET4[StatsHdsList[hdrCnt]] = H1_RR4_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR5AET1[StatsHdsList[hdrCnt]] = H1_RR5_AET1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR5AET2[StatsHdsList[hdrCnt]] = H1_RR5_AET2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR5AET3[StatsHdsList[hdrCnt]] = H1_RR5_AET3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR5AET4[StatsHdsList[hdrCnt]] = H1_RR5_AET4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR1RO1[StatsHdsList[hdrCnt]] = H1_HR1_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR1RO2[StatsHdsList[hdrCnt]] = H1_HR1_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR1RO3[StatsHdsList[hdrCnt]] = H1_HR1_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR1RO4[StatsHdsList[hdrCnt]] = H1_HR1_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR2RO1[StatsHdsList[hdrCnt]] = H1_HR2_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR2RO2[StatsHdsList[hdrCnt]] = H1_HR2_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR2RO3[StatsHdsList[hdrCnt]] = H1_HR2_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR2RO4[StatsHdsList[hdrCnt]] = H1_HR2_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR1RO1[StatsHdsList[hdrCnt]] = H1_RR1_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR1RO2[StatsHdsList[hdrCnt]] = H1_RR1_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR1RO3[StatsHdsList[hdrCnt]] = H1_RR1_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR1RO4[StatsHdsList[hdrCnt]] = H1_RR1_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR2RO1[StatsHdsList[hdrCnt]] = H1_RR2_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR2RO2[StatsHdsList[hdrCnt]] = H1_RR2_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR2RO3[StatsHdsList[hdrCnt]] = H1_RR2_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR2RO4[StatsHdsList[hdrCnt]] = H1_RR2_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR3RO1[StatsHdsList[hdrCnt]] = H1_RR3_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR3RO2[StatsHdsList[hdrCnt]] = H1_RR3_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR3RO3[StatsHdsList[hdrCnt]] = H1_RR3_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR3RO4[StatsHdsList[hdrCnt]] = H1_RR3_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR4RO1[StatsHdsList[hdrCnt]] = H1_RR4_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR4RO2[StatsHdsList[hdrCnt]] = H1_RR4_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR4RO3[StatsHdsList[hdrCnt]] = H1_RR4_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR4RO4[StatsHdsList[hdrCnt]] = H1_RR4_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR5RO1[StatsHdsList[hdrCnt]] = H1_RR5_RO1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR5RO2[StatsHdsList[hdrCnt]] = H1_RR5_RO2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR5RO3[StatsHdsList[hdrCnt]] = H1_RR5_RO3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR5RO4[StatsHdsList[hdrCnt]] = H1_RR5_RO4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR1RE1[StatsHdsList[hdrCnt]] = H1_HR1_RE1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR1RE2[StatsHdsList[hdrCnt]] = H1_HR1_RE2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR1RE3[StatsHdsList[hdrCnt]] = H1_HR1_RE3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR1RE4[StatsHdsList[hdrCnt]] = H1_HR1_RE4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR2RE1[StatsHdsList[hdrCnt]] = H1_HR2_RE1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR2RE2[StatsHdsList[hdrCnt]] = H1_HR2_RE2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR2RE3[StatsHdsList[hdrCnt]] = H1_HR2_RE3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictHR2RE4[StatsHdsList[hdrCnt]] = H1_HR2_RE4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR1RE1[StatsHdsList[hdrCnt]] = H1_RR1_RE1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR1RE2[StatsHdsList[hdrCnt]] = H1_RR1_RE2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR1RE3[StatsHdsList[hdrCnt]] = H1_RR1_RE3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR1RE4[StatsHdsList[hdrCnt]] = H1_RR1_RE4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR2RE1[StatsHdsList[hdrCnt]] = H1_RR2_RE1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR2RE2[StatsHdsList[hdrCnt]] = H1_RR2_RE2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR2RE3[StatsHdsList[hdrCnt]] = H1_RR2_RE3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR2RE4[StatsHdsList[hdrCnt]] = H1_RR2_RE4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR3RE1[StatsHdsList[hdrCnt]] = H1_RR3_RE1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR3RE2[StatsHdsList[hdrCnt]] = H1_RR3_RE2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR3RE3[StatsHdsList[hdrCnt]] = H1_RR3_RE3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR3RE4[StatsHdsList[hdrCnt]] = H1_RR3_RE4.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR4RE1[StatsHdsList[hdrCnt]] = H1_RR4_RE1.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR4RE2[StatsHdsList[hdrCnt]] = H1_RR4_RE2.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR4RE3[StatsHdsList[hdrCnt]] = H1_RR4_RE3.quantile( q=curQ, axis=1 ).to_numpy()
    H1_DictRR4RE4[StatsHdsList[hdrCnt]] = H1_RR4_RE4.quantile( q=curQ, axis=1 ).to_numpy()
    hdrCnt += 1
# end for
H1_DictHR1AET1[StatsHdsList[hdrCnt]] = H1_HR1_AET1.mean( axis=1 ).to_numpy()
H1_DictHR1AET2[StatsHdsList[hdrCnt]] = H1_HR1_AET2.mean( axis=1 ).to_numpy()
H1_DictHR1AET3[StatsHdsList[hdrCnt]] = H1_HR1_AET3.mean( axis=1 ).to_numpy()
H1_DictHR1AET4[StatsHdsList[hdrCnt]] = H1_HR1_AET4.mean( axis=1 ).to_numpy()
H1_DictHR2AET1[StatsHdsList[hdrCnt]] = H1_HR2_AET1.mean( axis=1 ).to_numpy()
H1_DictHR2AET2[StatsHdsList[hdrCnt]] = H1_HR2_AET2.mean( axis=1 ).to_numpy()
H1_DictHR2AET3[StatsHdsList[hdrCnt]] = H1_HR2_AET3.mean( axis=1 ).to_numpy()
H1_DictHR2AET4[StatsHdsList[hdrCnt]] = H1_HR2_AET4.mean( axis=1 ).to_numpy()
H1_DictRR1AET1[StatsHdsList[hdrCnt]] = H1_RR1_AET1.mean( axis=1 ).to_numpy()
H1_DictRR1AET2[StatsHdsList[hdrCnt]] = H1_RR1_AET2.mean( axis=1 ).to_numpy()
H1_DictRR1AET3[StatsHdsList[hdrCnt]] = H1_RR1_AET3.mean( axis=1 ).to_numpy()
H1_DictRR1AET4[StatsHdsList[hdrCnt]] = H1_RR1_AET4.mean( axis=1 ).to_numpy()
H1_DictRR2AET1[StatsHdsList[hdrCnt]] = H1_RR2_AET1.mean( axis=1 ).to_numpy()
H1_DictRR2AET2[StatsHdsList[hdrCnt]] = H1_RR2_AET2.mean( axis=1 ).to_numpy()
H1_DictRR2AET3[StatsHdsList[hdrCnt]] = H1_RR2_AET3.mean( axis=1 ).to_numpy()
H1_DictRR2AET4[StatsHdsList[hdrCnt]] = H1_RR2_AET4.mean( axis=1 ).to_numpy()
H1_DictRR3AET1[StatsHdsList[hdrCnt]] = H1_RR3_AET1.mean( axis=1 ).to_numpy()
H1_DictRR3AET2[StatsHdsList[hdrCnt]] = H1_RR3_AET2.mean( axis=1 ).to_numpy()
H1_DictRR3AET3[StatsHdsList[hdrCnt]] = H1_RR3_AET3.mean( axis=1 ).to_numpy()
H1_DictRR3AET4[StatsHdsList[hdrCnt]] = H1_RR3_AET4.mean( axis=1 ).to_numpy()
H1_DictRR4AET1[StatsHdsList[hdrCnt]] = H1_RR4_AET1.mean( axis=1 ).to_numpy()
H1_DictRR4AET2[StatsHdsList[hdrCnt]] = H1_RR4_AET2.mean( axis=1 ).to_numpy()
H1_DictRR4AET3[StatsHdsList[hdrCnt]] = H1_RR4_AET3.mean( axis=1 ).to_numpy()
H1_DictRR4AET4[StatsHdsList[hdrCnt]] = H1_RR4_AET4.mean( axis=1 ).to_numpy()
H1_DictRR5AET1[StatsHdsList[hdrCnt]] = H1_RR5_AET1.mean( axis=1 ).to_numpy()
H1_DictRR5AET2[StatsHdsList[hdrCnt]] = H1_RR5_AET2.mean( axis=1 ).to_numpy()
H1_DictRR5AET3[StatsHdsList[hdrCnt]] = H1_RR5_AET3.mean( axis=1 ).to_numpy()
H1_DictRR5AET4[StatsHdsList[hdrCnt]] = H1_RR5_AET4.mean( axis=1 ).to_numpy()
H1_DictHR1RO1[StatsHdsList[hdrCnt]] = H1_HR1_RO1.mean( axis=1 ).to_numpy()
H1_DictHR1RO2[StatsHdsList[hdrCnt]] = H1_HR1_RO2.mean( axis=1 ).to_numpy()
H1_DictHR1RO3[StatsHdsList[hdrCnt]] = H1_HR1_RO3.mean( axis=1 ).to_numpy()
H1_DictHR1RO4[StatsHdsList[hdrCnt]] = H1_HR1_RO4.mean( axis=1 ).to_numpy()
H1_DictHR2RO1[StatsHdsList[hdrCnt]] = H1_HR2_RO1.mean( axis=1 ).to_numpy()
H1_DictHR2RO2[StatsHdsList[hdrCnt]] = H1_HR2_RO2.mean( axis=1 ).to_numpy()
H1_DictHR2RO3[StatsHdsList[hdrCnt]] = H1_HR2_RO3.mean( axis=1 ).to_numpy()
H1_DictHR2RO4[StatsHdsList[hdrCnt]] = H1_HR2_RO4.mean( axis=1 ).to_numpy()
H1_DictRR1RO1[StatsHdsList[hdrCnt]] = H1_RR1_RO1.mean( axis=1 ).to_numpy()
H1_DictRR1RO2[StatsHdsList[hdrCnt]] = H1_RR1_RO2.mean( axis=1 ).to_numpy()
H1_DictRR1RO3[StatsHdsList[hdrCnt]] = H1_RR1_RO3.mean( axis=1 ).to_numpy()
H1_DictRR1RO4[StatsHdsList[hdrCnt]] = H1_RR1_RO4.mean( axis=1 ).to_numpy()
H1_DictRR2RO1[StatsHdsList[hdrCnt]] = H1_RR2_RO1.mean( axis=1 ).to_numpy()
H1_DictRR2RO2[StatsHdsList[hdrCnt]] = H1_RR2_RO2.mean( axis=1 ).to_numpy()
H1_DictRR2RO3[StatsHdsList[hdrCnt]] = H1_RR2_RO3.mean( axis=1 ).to_numpy()
H1_DictRR2RO4[StatsHdsList[hdrCnt]] = H1_RR2_RO4.mean( axis=1 ).to_numpy()
H1_DictRR3RO1[StatsHdsList[hdrCnt]] = H1_RR3_RO1.mean( axis=1 ).to_numpy()
H1_DictRR3RO2[StatsHdsList[hdrCnt]] = H1_RR3_RO2.mean( axis=1 ).to_numpy()
H1_DictRR3RO3[StatsHdsList[hdrCnt]] = H1_RR3_RO3.mean( axis=1 ).to_numpy()
H1_DictRR3RO4[StatsHdsList[hdrCnt]] = H1_RR3_RO4.mean( axis=1 ).to_numpy()
H1_DictRR4RO1[StatsHdsList[hdrCnt]] = H1_RR4_RO1.mean( axis=1 ).to_numpy()
H1_DictRR4RO2[StatsHdsList[hdrCnt]] = H1_RR4_RO2.mean( axis=1 ).to_numpy()
H1_DictRR4RO3[StatsHdsList[hdrCnt]] = H1_RR4_RO3.mean( axis=1 ).to_numpy()
H1_DictRR4RO4[StatsHdsList[hdrCnt]] = H1_RR4_RO4.mean( axis=1 ).to_numpy()
H1_DictRR5RO1[StatsHdsList[hdrCnt]] = H1_RR5_RO1.mean( axis=1 ).to_numpy()
H1_DictRR5RO2[StatsHdsList[hdrCnt]] = H1_RR5_RO2.mean( axis=1 ).to_numpy()
H1_DictRR5RO3[StatsHdsList[hdrCnt]] = H1_RR5_RO3.mean( axis=1 ).to_numpy()
H1_DictRR5RO4[StatsHdsList[hdrCnt]] = H1_RR5_RO4.mean( axis=1 ).to_numpy()
H1_DictHR1RE1[StatsHdsList[hdrCnt]] = H1_HR1_RE1.mean( axis=1 ).to_numpy()
H1_DictHR1RE2[StatsHdsList[hdrCnt]] = H1_HR1_RE2.mean( axis=1 ).to_numpy()
H1_DictHR1RE3[StatsHdsList[hdrCnt]] = H1_HR1_RE3.mean( axis=1 ).to_numpy()
H1_DictHR1RE4[StatsHdsList[hdrCnt]] = H1_HR1_RE4.mean( axis=1 ).to_numpy()
H1_DictHR2RE1[StatsHdsList[hdrCnt]] = H1_HR2_RE1.mean( axis=1 ).to_numpy()
H1_DictHR2RE2[StatsHdsList[hdrCnt]] = H1_HR2_RE2.mean( axis=1 ).to_numpy()
H1_DictHR2RE3[StatsHdsList[hdrCnt]] = H1_HR2_RE3.mean( axis=1 ).to_numpy()
H1_DictHR2RE4[StatsHdsList[hdrCnt]] = H1_HR2_RE4.mean( axis=1 ).to_numpy()
H1_DictRR1RE1[StatsHdsList[hdrCnt]] = H1_RR1_RE1.mean( axis=1 ).to_numpy()
H1_DictRR1RE2[StatsHdsList[hdrCnt]] = H1_RR1_RE2.mean( axis=1 ).to_numpy()
H1_DictRR1RE3[StatsHdsList[hdrCnt]] = H1_RR1_RE3.mean( axis=1 ).to_numpy()
H1_DictRR1RE4[StatsHdsList[hdrCnt]] = H1_RR1_RE4.mean( axis=1 ).to_numpy()
H1_DictRR2RE1[StatsHdsList[hdrCnt]] = H1_RR2_RE1.mean( axis=1 ).to_numpy()
H1_DictRR2RE2[StatsHdsList[hdrCnt]] = H1_RR2_RE2.mean( axis=1 ).to_numpy()
H1_DictRR2RE3[StatsHdsList[hdrCnt]] = H1_RR2_RE3.mean( axis=1 ).to_numpy()
H1_DictRR2RE4[StatsHdsList[hdrCnt]] = H1_RR2_RE4.mean( axis=1 ).to_numpy()
H1_DictRR3RE1[StatsHdsList[hdrCnt]] = H1_RR3_RE1.mean( axis=1 ).to_numpy()
H1_DictRR3RE2[StatsHdsList[hdrCnt]] = H1_RR3_RE2.mean( axis=1 ).to_numpy()
H1_DictRR3RE3[StatsHdsList[hdrCnt]] = H1_RR3_RE3.mean( axis=1 ).to_numpy()
H1_DictRR3RE4[StatsHdsList[hdrCnt]] = H1_RR3_RE4.mean( axis=1 ).to_numpy()
H1_DictRR4RE1[StatsHdsList[hdrCnt]] = H1_RR4_RE1.mean( axis=1 ).to_numpy()
H1_DictRR4RE2[StatsHdsList[hdrCnt]] = H1_RR4_RE2.mean( axis=1 ).to_numpy()
H1_DictRR4RE3[StatsHdsList[hdrCnt]] = H1_RR4_RE3.mean( axis=1 ).to_numpy()
H1_DictRR4RE4[StatsHdsList[hdrCnt]] = H1_RR4_RE4.mean( axis=1 ).to_numpy()
hdrCnt += 1

In [35]:
H1_DictHR1AET1[StatsHdsList[hdrCnt]] = H1_HR1_AET1.var( axis=1 ).to_numpy()
H1_DictHR1AET2[StatsHdsList[hdrCnt]] = H1_HR1_AET2.var( axis=1 ).to_numpy()
H1_DictHR1AET3[StatsHdsList[hdrCnt]] = H1_HR1_AET3.var( axis=1 ).to_numpy()
H1_DictHR1AET4[StatsHdsList[hdrCnt]] = H1_HR1_AET4.var( axis=1 ).to_numpy()
H1_DictHR2AET1[StatsHdsList[hdrCnt]] = H1_HR2_AET1.var( axis=1 ).to_numpy()
H1_DictHR2AET2[StatsHdsList[hdrCnt]] = H1_HR2_AET2.var( axis=1 ).to_numpy()
H1_DictHR2AET3[StatsHdsList[hdrCnt]] = H1_HR2_AET3.var( axis=1 ).to_numpy()
H1_DictHR2AET4[StatsHdsList[hdrCnt]] = H1_HR2_AET4.var( axis=1 ).to_numpy()
H1_DictRR1AET1[StatsHdsList[hdrCnt]] = H1_RR1_AET1.var( axis=1 ).to_numpy()
H1_DictRR1AET2[StatsHdsList[hdrCnt]] = H1_RR1_AET2.var( axis=1 ).to_numpy()
H1_DictRR1AET3[StatsHdsList[hdrCnt]] = H1_RR1_AET3.var( axis=1 ).to_numpy()
H1_DictRR1AET4[StatsHdsList[hdrCnt]] = H1_RR1_AET4.var( axis=1 ).to_numpy()
H1_DictRR2AET1[StatsHdsList[hdrCnt]] = H1_RR2_AET1.var( axis=1 ).to_numpy()
H1_DictRR2AET2[StatsHdsList[hdrCnt]] = H1_RR2_AET2.var( axis=1 ).to_numpy()
H1_DictRR2AET3[StatsHdsList[hdrCnt]] = H1_RR2_AET3.var( axis=1 ).to_numpy()
H1_DictRR2AET4[StatsHdsList[hdrCnt]] = H1_RR2_AET4.var( axis=1 ).to_numpy()
H1_DictRR3AET1[StatsHdsList[hdrCnt]] = H1_RR3_AET1.var( axis=1 ).to_numpy()
H1_DictRR3AET2[StatsHdsList[hdrCnt]] = H1_RR3_AET2.var( axis=1 ).to_numpy()
H1_DictRR3AET3[StatsHdsList[hdrCnt]] = H1_RR3_AET3.var( axis=1 ).to_numpy()
H1_DictRR3AET4[StatsHdsList[hdrCnt]] = H1_RR3_AET4.var( axis=1 ).to_numpy()
H1_DictRR4AET1[StatsHdsList[hdrCnt]] = H1_RR4_AET1.var( axis=1 ).to_numpy()
H1_DictRR4AET2[StatsHdsList[hdrCnt]] = H1_RR4_AET2.var( axis=1 ).to_numpy()
H1_DictRR4AET3[StatsHdsList[hdrCnt]] = H1_RR4_AET3.var( axis=1 ).to_numpy()
H1_DictRR4AET4[StatsHdsList[hdrCnt]] = H1_RR4_AET4.var( axis=1 ).to_numpy()
H1_DictRR5AET1[StatsHdsList[hdrCnt]] = H1_RR5_AET1.var( axis=1 ).to_numpy()
H1_DictRR5AET2[StatsHdsList[hdrCnt]] = H1_RR5_AET2.var( axis=1 ).to_numpy()
H1_DictRR5AET3[StatsHdsList[hdrCnt]] = H1_RR5_AET3.var( axis=1 ).to_numpy()
H1_DictRR5AET4[StatsHdsList[hdrCnt]] = H1_RR5_AET4.var( axis=1 ).to_numpy()
H1_DictHR1RO1[StatsHdsList[hdrCnt]] = H1_HR1_RO1.var( axis=1 ).to_numpy()
H1_DictHR1RO2[StatsHdsList[hdrCnt]] = H1_HR1_RO2.var( axis=1 ).to_numpy()
H1_DictHR1RO3[StatsHdsList[hdrCnt]] = H1_HR1_RO3.var( axis=1 ).to_numpy()
H1_DictHR1RO4[StatsHdsList[hdrCnt]] = H1_HR1_RO4.var( axis=1 ).to_numpy()
H1_DictHR2RO1[StatsHdsList[hdrCnt]] = H1_HR2_RO1.var( axis=1 ).to_numpy()
H1_DictHR2RO2[StatsHdsList[hdrCnt]] = H1_HR2_RO2.var( axis=1 ).to_numpy()
H1_DictHR2RO3[StatsHdsList[hdrCnt]] = H1_HR2_RO3.var( axis=1 ).to_numpy()
H1_DictHR2RO4[StatsHdsList[hdrCnt]] = H1_HR2_RO4.var( axis=1 ).to_numpy()
H1_DictRR1RO1[StatsHdsList[hdrCnt]] = H1_RR1_RO1.var( axis=1 ).to_numpy()
H1_DictRR1RO2[StatsHdsList[hdrCnt]] = H1_RR1_RO2.var( axis=1 ).to_numpy()
H1_DictRR1RO3[StatsHdsList[hdrCnt]] = H1_RR1_RO3.var( axis=1 ).to_numpy()
H1_DictRR1RO4[StatsHdsList[hdrCnt]] = H1_RR1_RO4.var( axis=1 ).to_numpy()
H1_DictRR2RO1[StatsHdsList[hdrCnt]] = H1_RR2_RO1.var( axis=1 ).to_numpy()
H1_DictRR2RO2[StatsHdsList[hdrCnt]] = H1_RR2_RO2.var( axis=1 ).to_numpy()
H1_DictRR2RO3[StatsHdsList[hdrCnt]] = H1_RR2_RO3.var( axis=1 ).to_numpy()
H1_DictRR2RO4[StatsHdsList[hdrCnt]] = H1_RR2_RO4.var( axis=1 ).to_numpy()
H1_DictRR3RO1[StatsHdsList[hdrCnt]] = H1_RR3_RO1.var( axis=1 ).to_numpy()
H1_DictRR3RO2[StatsHdsList[hdrCnt]] = H1_RR3_RO2.var( axis=1 ).to_numpy()
H1_DictRR3RO3[StatsHdsList[hdrCnt]] = H1_RR3_RO3.var( axis=1 ).to_numpy()
H1_DictRR3RO4[StatsHdsList[hdrCnt]] = H1_RR3_RO4.var( axis=1 ).to_numpy()
H1_DictRR4RO1[StatsHdsList[hdrCnt]] = H1_RR4_RO1.var( axis=1 ).to_numpy()
H1_DictRR4RO2[StatsHdsList[hdrCnt]] = H1_RR4_RO2.var( axis=1 ).to_numpy()
H1_DictRR4RO3[StatsHdsList[hdrCnt]] = H1_RR4_RO3.var( axis=1 ).to_numpy()
H1_DictRR4RO4[StatsHdsList[hdrCnt]] = H1_RR4_RO4.var( axis=1 ).to_numpy()
H1_DictRR5RO1[StatsHdsList[hdrCnt]] = H1_RR5_RO1.var( axis=1 ).to_numpy()
H1_DictRR5RO2[StatsHdsList[hdrCnt]] = H1_RR5_RO2.var( axis=1 ).to_numpy()
H1_DictRR5RO3[StatsHdsList[hdrCnt]] = H1_RR5_RO3.var( axis=1 ).to_numpy()
H1_DictRR5RO4[StatsHdsList[hdrCnt]] = H1_RR5_RO4.var( axis=1 ).to_numpy()
H1_DictHR1RE1[StatsHdsList[hdrCnt]] = H1_HR1_RE1.var( axis=1 ).to_numpy()
H1_DictHR1RE2[StatsHdsList[hdrCnt]] = H1_HR1_RE2.var( axis=1 ).to_numpy()
H1_DictHR1RE3[StatsHdsList[hdrCnt]] = H1_HR1_RE3.var( axis=1 ).to_numpy()
H1_DictHR1RE4[StatsHdsList[hdrCnt]] = H1_HR1_RE4.var( axis=1 ).to_numpy()
H1_DictHR2RE1[StatsHdsList[hdrCnt]] = H1_HR2_RE1.var( axis=1 ).to_numpy()
H1_DictHR2RE2[StatsHdsList[hdrCnt]] = H1_HR2_RE2.var( axis=1 ).to_numpy()
H1_DictHR2RE3[StatsHdsList[hdrCnt]] = H1_HR2_RE3.var( axis=1 ).to_numpy()
H1_DictHR2RE4[StatsHdsList[hdrCnt]] = H1_HR2_RE4.var( axis=1 ).to_numpy()
H1_DictRR1RE1[StatsHdsList[hdrCnt]] = H1_RR1_RE1.var( axis=1 ).to_numpy()
H1_DictRR1RE2[StatsHdsList[hdrCnt]] = H1_RR1_RE2.var( axis=1 ).to_numpy()
H1_DictRR1RE3[StatsHdsList[hdrCnt]] = H1_RR1_RE3.var( axis=1 ).to_numpy()
H1_DictRR1RE4[StatsHdsList[hdrCnt]] = H1_RR1_RE4.var( axis=1 ).to_numpy()
H1_DictRR2RE1[StatsHdsList[hdrCnt]] = H1_RR2_RE1.var( axis=1 ).to_numpy()
H1_DictRR2RE2[StatsHdsList[hdrCnt]] = H1_RR2_RE2.var( axis=1 ).to_numpy()
H1_DictRR2RE3[StatsHdsList[hdrCnt]] = H1_RR2_RE3.var( axis=1 ).to_numpy()
H1_DictRR2RE4[StatsHdsList[hdrCnt]] = H1_RR2_RE4.var( axis=1 ).to_numpy()
H1_DictRR3RE1[StatsHdsList[hdrCnt]] = H1_RR3_RE1.var( axis=1 ).to_numpy()
H1_DictRR3RE2[StatsHdsList[hdrCnt]] = H1_RR3_RE2.var( axis=1 ).to_numpy()
H1_DictRR3RE3[StatsHdsList[hdrCnt]] = H1_RR3_RE3.var( axis=1 ).to_numpy()
H1_DictRR3RE4[StatsHdsList[hdrCnt]] = H1_RR3_RE4.var( axis=1 ).to_numpy()
H1_DictRR4RE1[StatsHdsList[hdrCnt]] = H1_RR4_RE1.var( axis=1 ).to_numpy()
H1_DictRR4RE2[StatsHdsList[hdrCnt]] = H1_RR4_RE2.var( axis=1 ).to_numpy()
H1_DictRR4RE3[StatsHdsList[hdrCnt]] = H1_RR4_RE3.var( axis=1 ).to_numpy()
H1_DictRR4RE4[StatsHdsList[hdrCnt]] = H1_RR4_RE4.var( axis=1 ).to_numpy()
hdrCnt += 1

In [36]:
H1_DictHR1AET1[StatsHdsList[hdrCnt]] = H1_HR1_AET1.skew( axis=1 ).to_numpy()
H1_DictHR1AET2[StatsHdsList[hdrCnt]] = H1_HR1_AET2.skew( axis=1 ).to_numpy()
H1_DictHR1AET3[StatsHdsList[hdrCnt]] = H1_HR1_AET3.skew( axis=1 ).to_numpy()
H1_DictHR1AET4[StatsHdsList[hdrCnt]] = H1_HR1_AET4.skew( axis=1 ).to_numpy()
H1_DictHR2AET1[StatsHdsList[hdrCnt]] = H1_HR2_AET1.skew( axis=1 ).to_numpy()
H1_DictHR2AET2[StatsHdsList[hdrCnt]] = H1_HR2_AET2.skew( axis=1 ).to_numpy()
H1_DictHR2AET3[StatsHdsList[hdrCnt]] = H1_HR2_AET3.skew( axis=1 ).to_numpy()
H1_DictHR2AET4[StatsHdsList[hdrCnt]] = H1_HR2_AET4.skew( axis=1 ).to_numpy()
H1_DictRR1AET1[StatsHdsList[hdrCnt]] = H1_RR1_AET1.skew( axis=1 ).to_numpy()
H1_DictRR1AET2[StatsHdsList[hdrCnt]] = H1_RR1_AET2.skew( axis=1 ).to_numpy()
H1_DictRR1AET3[StatsHdsList[hdrCnt]] = H1_RR1_AET3.skew( axis=1 ).to_numpy()
H1_DictRR1AET4[StatsHdsList[hdrCnt]] = H1_RR1_AET4.skew( axis=1 ).to_numpy()
H1_DictRR2AET1[StatsHdsList[hdrCnt]] = H1_RR2_AET1.skew( axis=1 ).to_numpy()
H1_DictRR2AET2[StatsHdsList[hdrCnt]] = H1_RR2_AET2.skew( axis=1 ).to_numpy()
H1_DictRR2AET3[StatsHdsList[hdrCnt]] = H1_RR2_AET3.skew( axis=1 ).to_numpy()
H1_DictRR2AET4[StatsHdsList[hdrCnt]] = H1_RR2_AET4.skew( axis=1 ).to_numpy()
H1_DictRR3AET1[StatsHdsList[hdrCnt]] = H1_RR3_AET1.skew( axis=1 ).to_numpy()
H1_DictRR3AET2[StatsHdsList[hdrCnt]] = H1_RR3_AET2.skew( axis=1 ).to_numpy()
H1_DictRR3AET3[StatsHdsList[hdrCnt]] = H1_RR3_AET3.skew( axis=1 ).to_numpy()
H1_DictRR3AET4[StatsHdsList[hdrCnt]] = H1_RR3_AET4.skew( axis=1 ).to_numpy()
H1_DictRR4AET1[StatsHdsList[hdrCnt]] = H1_RR4_AET1.skew( axis=1 ).to_numpy()
H1_DictRR4AET2[StatsHdsList[hdrCnt]] = H1_RR4_AET2.skew( axis=1 ).to_numpy()
H1_DictRR4AET3[StatsHdsList[hdrCnt]] = H1_RR4_AET3.skew( axis=1 ).to_numpy()
H1_DictRR4AET4[StatsHdsList[hdrCnt]] = H1_RR4_AET4.skew( axis=1 ).to_numpy()
H1_DictRR5AET1[StatsHdsList[hdrCnt]] = H1_RR5_AET1.skew( axis=1 ).to_numpy()
H1_DictRR5AET2[StatsHdsList[hdrCnt]] = H1_RR5_AET2.skew( axis=1 ).to_numpy()
H1_DictRR5AET3[StatsHdsList[hdrCnt]] = H1_RR5_AET3.skew( axis=1 ).to_numpy()
H1_DictRR5AET4[StatsHdsList[hdrCnt]] = H1_RR5_AET4.skew( axis=1 ).to_numpy()
H1_DictHR1RO1[StatsHdsList[hdrCnt]] = H1_HR1_RO1.skew( axis=1 ).to_numpy()
H1_DictHR1RO2[StatsHdsList[hdrCnt]] = H1_HR1_RO2.skew( axis=1 ).to_numpy()
H1_DictHR1RO3[StatsHdsList[hdrCnt]] = H1_HR1_RO3.skew( axis=1 ).to_numpy()
H1_DictHR1RO4[StatsHdsList[hdrCnt]] = H1_HR1_RO4.skew( axis=1 ).to_numpy()
H1_DictHR2RO1[StatsHdsList[hdrCnt]] = H1_HR2_RO1.skew( axis=1 ).to_numpy()
H1_DictHR2RO2[StatsHdsList[hdrCnt]] = H1_HR2_RO2.skew( axis=1 ).to_numpy()
H1_DictHR2RO3[StatsHdsList[hdrCnt]] = H1_HR2_RO3.skew( axis=1 ).to_numpy()
H1_DictHR2RO4[StatsHdsList[hdrCnt]] = H1_HR2_RO4.skew( axis=1 ).to_numpy()
H1_DictRR1RO1[StatsHdsList[hdrCnt]] = H1_RR1_RO1.skew( axis=1 ).to_numpy()
H1_DictRR1RO2[StatsHdsList[hdrCnt]] = H1_RR1_RO2.skew( axis=1 ).to_numpy()
H1_DictRR1RO3[StatsHdsList[hdrCnt]] = H1_RR1_RO3.skew( axis=1 ).to_numpy()
H1_DictRR1RO4[StatsHdsList[hdrCnt]] = H1_RR1_RO4.skew( axis=1 ).to_numpy()
H1_DictRR2RO1[StatsHdsList[hdrCnt]] = H1_RR2_RO1.skew( axis=1 ).to_numpy()
H1_DictRR2RO2[StatsHdsList[hdrCnt]] = H1_RR2_RO2.skew( axis=1 ).to_numpy()
H1_DictRR2RO3[StatsHdsList[hdrCnt]] = H1_RR2_RO3.skew( axis=1 ).to_numpy()
H1_DictRR2RO4[StatsHdsList[hdrCnt]] = H1_RR2_RO4.skew( axis=1 ).to_numpy()
H1_DictRR3RO1[StatsHdsList[hdrCnt]] = H1_RR3_RO1.skew( axis=1 ).to_numpy()
H1_DictRR3RO2[StatsHdsList[hdrCnt]] = H1_RR3_RO2.skew( axis=1 ).to_numpy()
H1_DictRR3RO3[StatsHdsList[hdrCnt]] = H1_RR3_RO3.skew( axis=1 ).to_numpy()
H1_DictRR3RO4[StatsHdsList[hdrCnt]] = H1_RR3_RO4.skew( axis=1 ).to_numpy()
H1_DictRR4RO1[StatsHdsList[hdrCnt]] = H1_RR4_RO1.skew( axis=1 ).to_numpy()
H1_DictRR4RO2[StatsHdsList[hdrCnt]] = H1_RR4_RO2.skew( axis=1 ).to_numpy()
H1_DictRR4RO3[StatsHdsList[hdrCnt]] = H1_RR4_RO3.skew( axis=1 ).to_numpy()
H1_DictRR4RO4[StatsHdsList[hdrCnt]] = H1_RR4_RO4.skew( axis=1 ).to_numpy()
H1_DictRR5RO1[StatsHdsList[hdrCnt]] = H1_RR5_RO1.skew( axis=1 ).to_numpy()
H1_DictRR5RO2[StatsHdsList[hdrCnt]] = H1_RR5_RO2.skew( axis=1 ).to_numpy()
H1_DictRR5RO3[StatsHdsList[hdrCnt]] = H1_RR5_RO3.skew( axis=1 ).to_numpy()
H1_DictRR5RO4[StatsHdsList[hdrCnt]] = H1_RR5_RO4.skew( axis=1 ).to_numpy()
H1_DictHR1RE1[StatsHdsList[hdrCnt]] = H1_HR1_RE1.skew( axis=1 ).to_numpy()
H1_DictHR1RE2[StatsHdsList[hdrCnt]] = H1_HR1_RE2.skew( axis=1 ).to_numpy()
H1_DictHR1RE3[StatsHdsList[hdrCnt]] = H1_HR1_RE3.skew( axis=1 ).to_numpy()
H1_DictHR1RE4[StatsHdsList[hdrCnt]] = H1_HR1_RE4.skew( axis=1 ).to_numpy()
H1_DictHR2RE1[StatsHdsList[hdrCnt]] = H1_HR2_RE1.skew( axis=1 ).to_numpy()
H1_DictHR2RE2[StatsHdsList[hdrCnt]] = H1_HR2_RE2.skew( axis=1 ).to_numpy()
H1_DictHR2RE3[StatsHdsList[hdrCnt]] = H1_HR2_RE3.skew( axis=1 ).to_numpy()
H1_DictHR2RE4[StatsHdsList[hdrCnt]] = H1_HR2_RE4.skew( axis=1 ).to_numpy()
H1_DictRR1RE1[StatsHdsList[hdrCnt]] = H1_RR1_RE1.skew( axis=1 ).to_numpy()
H1_DictRR1RE2[StatsHdsList[hdrCnt]] = H1_RR1_RE2.skew( axis=1 ).to_numpy()
H1_DictRR1RE3[StatsHdsList[hdrCnt]] = H1_RR1_RE3.skew( axis=1 ).to_numpy()
H1_DictRR1RE4[StatsHdsList[hdrCnt]] = H1_RR1_RE4.skew( axis=1 ).to_numpy()
H1_DictRR2RE1[StatsHdsList[hdrCnt]] = H1_RR2_RE1.skew( axis=1 ).to_numpy()
H1_DictRR2RE2[StatsHdsList[hdrCnt]] = H1_RR2_RE2.skew( axis=1 ).to_numpy()
H1_DictRR2RE3[StatsHdsList[hdrCnt]] = H1_RR2_RE3.skew( axis=1 ).to_numpy()
H1_DictRR2RE4[StatsHdsList[hdrCnt]] = H1_RR2_RE4.skew( axis=1 ).to_numpy()
H1_DictRR3RE1[StatsHdsList[hdrCnt]] = H1_RR3_RE1.skew( axis=1 ).to_numpy()
H1_DictRR3RE2[StatsHdsList[hdrCnt]] = H1_RR3_RE2.skew( axis=1 ).to_numpy()
H1_DictRR3RE3[StatsHdsList[hdrCnt]] = H1_RR3_RE3.skew( axis=1 ).to_numpy()
H1_DictRR3RE4[StatsHdsList[hdrCnt]] = H1_RR3_RE4.skew( axis=1 ).to_numpy()
H1_DictRR4RE1[StatsHdsList[hdrCnt]] = H1_RR4_RE1.skew( axis=1 ).to_numpy()
H1_DictRR4RE2[StatsHdsList[hdrCnt]] = H1_RR4_RE2.skew( axis=1 ).to_numpy()
H1_DictRR4RE3[StatsHdsList[hdrCnt]] = H1_RR4_RE3.skew( axis=1 ).to_numpy()
H1_DictRR4RE4[StatsHdsList[hdrCnt]] = H1_RR4_RE4.skew( axis=1 ).to_numpy()
hdrCnt += 1

In [37]:
H1_DictHR1AET1[StatsHdsList[hdrCnt]] = H1_HR1_AET1.kurt( axis=1 ).to_numpy()
H1_DictHR1AET2[StatsHdsList[hdrCnt]] = H1_HR1_AET2.kurt( axis=1 ).to_numpy()
H1_DictHR1AET3[StatsHdsList[hdrCnt]] = H1_HR1_AET3.kurt( axis=1 ).to_numpy()
H1_DictHR1AET4[StatsHdsList[hdrCnt]] = H1_HR1_AET4.kurt( axis=1 ).to_numpy()
H1_DictHR2AET1[StatsHdsList[hdrCnt]] = H1_HR2_AET1.kurt( axis=1 ).to_numpy()
H1_DictHR2AET2[StatsHdsList[hdrCnt]] = H1_HR2_AET2.kurt( axis=1 ).to_numpy()
H1_DictHR2AET3[StatsHdsList[hdrCnt]] = H1_HR2_AET3.kurt( axis=1 ).to_numpy()
H1_DictHR2AET4[StatsHdsList[hdrCnt]] = H1_HR2_AET4.kurt( axis=1 ).to_numpy()
H1_DictRR1AET1[StatsHdsList[hdrCnt]] = H1_RR1_AET1.kurt( axis=1 ).to_numpy()
H1_DictRR1AET2[StatsHdsList[hdrCnt]] = H1_RR1_AET2.kurt( axis=1 ).to_numpy()
H1_DictRR1AET3[StatsHdsList[hdrCnt]] = H1_RR1_AET3.kurt( axis=1 ).to_numpy()
H1_DictRR1AET4[StatsHdsList[hdrCnt]] = H1_RR1_AET4.kurt( axis=1 ).to_numpy()
H1_DictRR2AET1[StatsHdsList[hdrCnt]] = H1_RR2_AET1.kurt( axis=1 ).to_numpy()
H1_DictRR2AET2[StatsHdsList[hdrCnt]] = H1_RR2_AET2.kurt( axis=1 ).to_numpy()
H1_DictRR2AET3[StatsHdsList[hdrCnt]] = H1_RR2_AET3.kurt( axis=1 ).to_numpy()
H1_DictRR2AET4[StatsHdsList[hdrCnt]] = H1_RR2_AET4.kurt( axis=1 ).to_numpy()
H1_DictRR3AET1[StatsHdsList[hdrCnt]] = H1_RR3_AET1.kurt( axis=1 ).to_numpy()
H1_DictRR3AET2[StatsHdsList[hdrCnt]] = H1_RR3_AET2.kurt( axis=1 ).to_numpy()
H1_DictRR3AET3[StatsHdsList[hdrCnt]] = H1_RR3_AET3.kurt( axis=1 ).to_numpy()
H1_DictRR3AET4[StatsHdsList[hdrCnt]] = H1_RR3_AET4.kurt( axis=1 ).to_numpy()
H1_DictRR4AET1[StatsHdsList[hdrCnt]] = H1_RR4_AET1.kurt( axis=1 ).to_numpy()
H1_DictRR4AET2[StatsHdsList[hdrCnt]] = H1_RR4_AET2.kurt( axis=1 ).to_numpy()
H1_DictRR4AET3[StatsHdsList[hdrCnt]] = H1_RR4_AET3.kurt( axis=1 ).to_numpy()
H1_DictRR4AET4[StatsHdsList[hdrCnt]] = H1_RR4_AET4.kurt( axis=1 ).to_numpy()
H1_DictRR5AET1[StatsHdsList[hdrCnt]] = H1_RR5_AET1.kurt( axis=1 ).to_numpy()
H1_DictRR5AET2[StatsHdsList[hdrCnt]] = H1_RR5_AET2.kurt( axis=1 ).to_numpy()
H1_DictRR5AET3[StatsHdsList[hdrCnt]] = H1_RR5_AET3.kurt( axis=1 ).to_numpy()
H1_DictRR5AET4[StatsHdsList[hdrCnt]] = H1_RR5_AET4.kurt( axis=1 ).to_numpy()
H1_DictHR1RO1[StatsHdsList[hdrCnt]] = H1_HR1_RO1.kurt( axis=1 ).to_numpy()
H1_DictHR1RO2[StatsHdsList[hdrCnt]] = H1_HR1_RO2.kurt( axis=1 ).to_numpy()
H1_DictHR1RO3[StatsHdsList[hdrCnt]] = H1_HR1_RO3.kurt( axis=1 ).to_numpy()
H1_DictHR1RO4[StatsHdsList[hdrCnt]] = H1_HR1_RO4.kurt( axis=1 ).to_numpy()
H1_DictHR2RO1[StatsHdsList[hdrCnt]] = H1_HR2_RO1.kurt( axis=1 ).to_numpy()
H1_DictHR2RO2[StatsHdsList[hdrCnt]] = H1_HR2_RO2.kurt( axis=1 ).to_numpy()
H1_DictHR2RO3[StatsHdsList[hdrCnt]] = H1_HR2_RO3.kurt( axis=1 ).to_numpy()
H1_DictHR2RO4[StatsHdsList[hdrCnt]] = H1_HR2_RO4.kurt( axis=1 ).to_numpy()
H1_DictRR1RO1[StatsHdsList[hdrCnt]] = H1_RR1_RO1.kurt( axis=1 ).to_numpy()
H1_DictRR1RO2[StatsHdsList[hdrCnt]] = H1_RR1_RO2.kurt( axis=1 ).to_numpy()
H1_DictRR1RO3[StatsHdsList[hdrCnt]] = H1_RR1_RO3.kurt( axis=1 ).to_numpy()
H1_DictRR1RO4[StatsHdsList[hdrCnt]] = H1_RR1_RO4.kurt( axis=1 ).to_numpy()
H1_DictRR2RO1[StatsHdsList[hdrCnt]] = H1_RR2_RO1.kurt( axis=1 ).to_numpy()
H1_DictRR2RO2[StatsHdsList[hdrCnt]] = H1_RR2_RO2.kurt( axis=1 ).to_numpy()
H1_DictRR2RO3[StatsHdsList[hdrCnt]] = H1_RR2_RO3.kurt( axis=1 ).to_numpy()
H1_DictRR2RO4[StatsHdsList[hdrCnt]] = H1_RR2_RO4.kurt( axis=1 ).to_numpy()
H1_DictRR3RO1[StatsHdsList[hdrCnt]] = H1_RR3_RO1.kurt( axis=1 ).to_numpy()
H1_DictRR3RO2[StatsHdsList[hdrCnt]] = H1_RR3_RO2.kurt( axis=1 ).to_numpy()
H1_DictRR3RO3[StatsHdsList[hdrCnt]] = H1_RR3_RO3.kurt( axis=1 ).to_numpy()
H1_DictRR3RO4[StatsHdsList[hdrCnt]] = H1_RR3_RO4.kurt( axis=1 ).to_numpy()
H1_DictRR4RO1[StatsHdsList[hdrCnt]] = H1_RR4_RO1.kurt( axis=1 ).to_numpy()
H1_DictRR4RO2[StatsHdsList[hdrCnt]] = H1_RR4_RO2.kurt( axis=1 ).to_numpy()
H1_DictRR4RO3[StatsHdsList[hdrCnt]] = H1_RR4_RO3.kurt( axis=1 ).to_numpy()
H1_DictRR4RO4[StatsHdsList[hdrCnt]] = H1_RR4_RO4.kurt( axis=1 ).to_numpy()
H1_DictRR5RO1[StatsHdsList[hdrCnt]] = H1_RR5_RO1.kurt( axis=1 ).to_numpy()
H1_DictRR5RO2[StatsHdsList[hdrCnt]] = H1_RR5_RO2.kurt( axis=1 ).to_numpy()
H1_DictRR5RO3[StatsHdsList[hdrCnt]] = H1_RR5_RO3.kurt( axis=1 ).to_numpy()
H1_DictRR5RO4[StatsHdsList[hdrCnt]] = H1_RR5_RO4.kurt( axis=1 ).to_numpy()
H1_DictHR1RE1[StatsHdsList[hdrCnt]] = H1_HR1_RE1.kurt( axis=1 ).to_numpy()
H1_DictHR1RE2[StatsHdsList[hdrCnt]] = H1_HR1_RE2.kurt( axis=1 ).to_numpy()
H1_DictHR1RE3[StatsHdsList[hdrCnt]] = H1_HR1_RE3.kurt( axis=1 ).to_numpy()
H1_DictHR1RE4[StatsHdsList[hdrCnt]] = H1_HR1_RE4.kurt( axis=1 ).to_numpy()
H1_DictHR2RE1[StatsHdsList[hdrCnt]] = H1_HR2_RE1.kurt( axis=1 ).to_numpy()
H1_DictHR2RE2[StatsHdsList[hdrCnt]] = H1_HR2_RE2.kurt( axis=1 ).to_numpy()
H1_DictHR2RE3[StatsHdsList[hdrCnt]] = H1_HR2_RE3.kurt( axis=1 ).to_numpy()
H1_DictHR2RE4[StatsHdsList[hdrCnt]] = H1_HR2_RE4.kurt( axis=1 ).to_numpy()
H1_DictRR1RE1[StatsHdsList[hdrCnt]] = H1_RR1_RE1.kurt( axis=1 ).to_numpy()
H1_DictRR1RE2[StatsHdsList[hdrCnt]] = H1_RR1_RE2.kurt( axis=1 ).to_numpy()
H1_DictRR1RE3[StatsHdsList[hdrCnt]] = H1_RR1_RE3.kurt( axis=1 ).to_numpy()
H1_DictRR1RE4[StatsHdsList[hdrCnt]] = H1_RR1_RE4.kurt( axis=1 ).to_numpy()
H1_DictRR2RE1[StatsHdsList[hdrCnt]] = H1_RR2_RE1.kurt( axis=1 ).to_numpy()
H1_DictRR2RE2[StatsHdsList[hdrCnt]] = H1_RR2_RE2.kurt( axis=1 ).to_numpy()
H1_DictRR2RE3[StatsHdsList[hdrCnt]] = H1_RR2_RE3.kurt( axis=1 ).to_numpy()
H1_DictRR2RE4[StatsHdsList[hdrCnt]] = H1_RR2_RE4.kurt( axis=1 ).to_numpy()
H1_DictRR3RE1[StatsHdsList[hdrCnt]] = H1_RR3_RE1.kurt( axis=1 ).to_numpy()
H1_DictRR3RE2[StatsHdsList[hdrCnt]] = H1_RR3_RE2.kurt( axis=1 ).to_numpy()
H1_DictRR3RE3[StatsHdsList[hdrCnt]] = H1_RR3_RE3.kurt( axis=1 ).to_numpy()
H1_DictRR3RE4[StatsHdsList[hdrCnt]] = H1_RR3_RE4.kurt( axis=1 ).to_numpy()
H1_DictRR4RE1[StatsHdsList[hdrCnt]] = H1_RR4_RE1.kurt( axis=1 ).to_numpy()
H1_DictRR4RE2[StatsHdsList[hdrCnt]] = H1_RR4_RE2.kurt( axis=1 ).to_numpy()
H1_DictRR4RE3[StatsHdsList[hdrCnt]] = H1_RR4_RE3.kurt( axis=1 ).to_numpy()
H1_DictRR4RE4[StatsHdsList[hdrCnt]] = H1_RR4_RE4.kurt( axis=1 ).to_numpy()

Make our DataFrames from the Stats

In [38]:
TimIndex1 = H1_HR1_AET1.index
TimIndex2 = H1_HR1_AET2.index
TimIndex3 = H1_HR1_AET3.index
TimIndex4 = H1_HR1_AET4.index

In [39]:
H1_HR1AETStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictHR1AET1 )
H1_HR1AETStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictHR1AET2 )
H1_HR1AETStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictHR1AET3 )
H1_HR1AETStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictHR1AET4 )
H1_HR2AETStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictHR2AET1 )
H1_HR2AETStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictHR2AET2 )
H1_HR2AETStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictHR2AET3 )
H1_HR2AETStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictHR2AET4 )
H1_RR1AETStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR1AET1 )
H1_RR1AETStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR1AET2 )
H1_RR1AETStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR1AET3 )
H1_RR1AETStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR1AET4 )
H1_RR2AETStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR2AET1 )
H1_RR2AETStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR2AET2 )
H1_RR2AETStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR2AET3 )
H1_RR2AETStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR2AET4 )
H1_RR3AETStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR3AET1 )
H1_RR3AETStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR3AET2 )
H1_RR3AETStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR3AET3 )
H1_RR3AETStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR3AET4 )
H1_RR4AETStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR4AET1 )
H1_RR4AETStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR4AET2 )
H1_RR4AETStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR4AET3 )
H1_RR4AETStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR4AET4 )
H1_RR5AETStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR5AET1 )
H1_RR5AETStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR5AET2 )
H1_RR5AETStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR5AET3 )
H1_RR5AETStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR5AET4 )

In [40]:
H1_HR1ROStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictHR1RO1 )
H1_HR1ROStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictHR1RO2 )
H1_HR1ROStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictHR1RO3 )
H1_HR1ROStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictHR1RO4 )
H1_HR2ROStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictHR2RO1 )
H1_HR2ROStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictHR2RO2 )
H1_HR2ROStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictHR2RO3 )
H1_HR2ROStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictHR2RO4 )
H1_RR1ROStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR1RO1 )
H1_RR1ROStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR1RO2 )
H1_RR1ROStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR1RO3 )
H1_RR1ROStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR1RO4 )
H1_RR2ROStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR2RO1 )
H1_RR2ROStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR2RO2 )
H1_RR2ROStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR2RO3 )
H1_RR2ROStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR2RO4 )
H1_RR3ROStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR3RO1 )
H1_RR3ROStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR3RO2 )
H1_RR3ROStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR3RO3 )
H1_RR3ROStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR3RO4 )
H1_RR4ROStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR4RO1 )
H1_RR4ROStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR4RO2 )
H1_RR4ROStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR4RO3 )
H1_RR4ROStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR4RO4 )
H1_RR5ROStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR5RO1 )
H1_RR5ROStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR5RO2 )
H1_RR5ROStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR5RO3 )
H1_RR5ROStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR5RO4 )

In [41]:
H1_HR1REStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictHR1RE1 )
H1_HR1REStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictHR1RE2 )
H1_HR1REStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictHR1RE3 )
H1_HR1REStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictHR1RE4 )
H1_HR2REStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictHR2RE1 )
H1_HR2REStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictHR2RE2 )
H1_HR2REStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictHR2RE3 )
H1_HR2REStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictHR2RE4 )
H1_RR1REStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR1RE1 )
H1_RR1REStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR1RE2 )
H1_RR1REStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR1RE3 )
H1_RR1REStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR1RE4 )
H1_RR2REStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR2RE1 )
H1_RR2REStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR2RE2 )
H1_RR2REStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR2RE3 )
H1_RR2REStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR2RE4 )
H1_RR3REStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR3RE1 )
H1_RR3REStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR3RE2 )
H1_RR3REStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR3RE3 )
H1_RR3REStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR3RE4 )
H1_RR4REStats1 = pd.DataFrame(index=TimIndex1, data=H1_DictRR4RE1 )
H1_RR4REStats2 = pd.DataFrame(index=TimIndex2, data=H1_DictRR4RE2 )
H1_RR4REStats3 = pd.DataFrame(index=TimIndex3, data=H1_DictRR4RE3 )
H1_RR4REStats4 = pd.DataFrame(index=TimIndex4, data=H1_DictRR4RE4 )

Output to a spreadsheets

In [42]:
OutFileName = "Cum_H1_HR1_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H1_HR1AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR1AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR1AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR1AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR1ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR1ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR1ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR1ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR1REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR1REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR1REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR1REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [43]:
OutFileName = "Cum_H1_HR2_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H1_HR2AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR2AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR2AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR2AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR2ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR2ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR2ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR2ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR2REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR2REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR2REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_HR2REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [44]:
OutFileName = "Cum_H1_RR1_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H1_RR1AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR1AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR1AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR1AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR1ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR1ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR1ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR1ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR1REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR1REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR1REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR1REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [45]:
OutFileName = "Cum_H1_RR2_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H1_RR2AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR2AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR2AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR2AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR2ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR2ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR2ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR2ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR2REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR2REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR2REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR2REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [46]:
OutFileName = "Cum_H1_RR3_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H1_RR3AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR3AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR3AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR3AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR3ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR3ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR3ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR3ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR3REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR3REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR3REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR3REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [47]:
OutFileName = "Cum_H1_RR4_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H1_RR4AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR4AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR4AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR4AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR4ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR4ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR4ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR4ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR4REStats1.to_excel( writer, sheet_name="RE_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR4REStats2.to_excel( writer, sheet_name="RE_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR4REStats3.to_excel( writer, sheet_name="RE_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR4REStats4.to_excel( writer, sheet_name="RE_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with

In [48]:
OutFileName = "Cum_H1_RR5_%s_ProcessMonthly_Stats.xlsx" % OUT_ROOT
OutFP = os.path.normpath( os.path.join( OUT_DIR, OutFileName ) )
with pd.ExcelWriter( OutFP ) as writer:
    H1_RR5AETStats1.to_excel( writer, sheet_name="AET_1", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR5AETStats2.to_excel( writer, sheet_name="AET_2", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR5AETStats3.to_excel( writer, sheet_name="AET_3", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR5AETStats4.to_excel( writer, sheet_name="AET_4", index_label="Date",
                              columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR5ROStats1.to_excel( writer, sheet_name="RO_1", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR5ROStats2.to_excel( writer, sheet_name="RO_2", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR5ROStats3.to_excel( writer, sheet_name="RO_3", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
    H1_RR5ROStats4.to_excel( writer, sheet_name="RO_4", index_label="Date",
                             columns=StatsHdsList, na_rep=str(np.nan) )
# end with